# Filter Itemid Prescriptions

First, we find the number of observation that has each unit of measurement. If the top frequent one has the observations less than 90% of total, then we discard it.

Also, there are many medicine that never be used by any patients, so we also discard it.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count

from utils import getConnection

%matplotlib inline

In [2]:
num_workers = cpu_count() // 2
res_dir = '/media/data/mimic/res'

In [3]:
conn = getConnection()
    
_adm = np.load(f'{res_dir}/admission_ids.npy', allow_pickle=True).tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load(f'{res_dir}/itemids.npy', allow_pickle=True).tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

In [4]:
def stat_prescript_unit_task(i, admission_ids_txt):
    conn = getConnection()
    # foreach medicine, list the dose unit of medicine and the count of observations that use that unit.
    cur = conn.cursor()
    cur.execute('SELECT dose_unit_rx, count(dose_unit_rx) FROM mimiciii.prescriptions WHERE formulary_drug_cd = \''+ str(i) +'\' and hadm_id in (select * from admission_ids) group by dose_unit_rx')
    outputunits = cur.fetchall()
    
    # sort it descendently
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    return (i, outputunits)

p = Pool(num_workers)
results = [p.apply_async(stat_prescript_unit_task, args=(i, admission_ids_txt)) for i in prescript_itemid]
p.close()
p.join()

results = [x.get() for x in results]
np.save(f'{res_dir}/filtered_prescript_raw.npy', {'raw': results})
print('saved!')

saved!


In [5]:
valid_prescript =[]
valid_prescript_unit =[]
dropped_id = []
notfound = []
results = np.load(f'{res_dir}/filtered_prescript_raw.npy', allow_pickle=True).tolist()['raw']

for x in results:
    i, outputunits = x[0], x[1]
    # check if medicine is never used by anybody, then discard it.
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): 
#         print("\n**PRES DROPPED "+str(i))
        notfound.append(i)
        for o in outputunits:
            print(o)
        print('')
        continue
        
    # calculate the percentage of observation of main unit.
    percentage = float(outputunits[0][1]) / total *100.
    if(percentage < 90): 
        # never drop the list A
#         if(i in manual_valid):
#             print("\n\n****PRES NOT DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
        
        # drop it !
#         else:
            dropped_id.append(i)
            print("\n\n****PRES DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
            continue
#     print("PRES "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits))
    
    # keep it and also save the main unit of it.
    valid_prescript.append(i)
    valid_prescript_unit.append(outputunits[0][0])




****PRES DROPPED SYRG1 : 75.53 : 6 : [('SYR', 3966), ('mL', 861), ('ml', 406), ('VIAL', 7), ('mg', 6), ('AMP', 5)]



****PRES DROPPED LIDO130I : 51.61 : 4 : [('ml', 16), ('mL', 14), ('dose', 1), (None, 0)]



****PRES DROPPED D5W500 : 60.14 : 3 : [('mL', 7475), ('ml', 4954), ('SYR', 1)]



****PRES DROPPED ETHYT : 75.00 : 2 : [('SPRY', 3), ('Appl', 1)]



****PRES DROPPED PCASYR : 64.52 : 3 : [('ml', 140), ('mL', 74), ('SYR', 3)]



****PRES DROPPED MPRED0 : 85.00 : 3 : [('mg', 17), ('dose', 2), ('MD to order daily dose', 1)]



****PRES DROPPED SYRINGEIND : 59.70 : 6 : [('ml', 40), ('dose', 10), ('mL', 8), ('SYR', 7), ('AMP', 1), ('VIAL', 1)]



****PRES DROPPED SEL25T : 63.64 : 2 : [('ml', 7), ('mL', 4)]



****PRES DROPPED DARB60SY : 50.00 : 3 : [('mcg/ml', 3), ('mcg/mL', 2), ('mcg', 1)]



****PRES DROPPED NSSYR : 88.24 : 2 : [('ml', 30), ('SYR', 4)]



****PRES DROPPED KCLBASE : 67.96 : 2 : [('mL', 6029), ('ml', 2842)]



****PRES DROPPED NS45500 : 54.17 : 2 : [('mL', 26), ('m

In [6]:
print(valid_prescript, valid_prescript_unit)
np.save(f'{res_dir}/filtered_prescript.npy',{'id':valid_prescript,'unit':valid_prescript_unit})
print('saved!')

['DXIG40I', 'NORESTFE', 'ARIP10', 'BACI5I', 'CALCITVITD', 'D12.5W250I', 'NEOLOPE2L', 'METR500I', 'LACO50L', 'OLME20', 'SILD25', 'MYCO250IND', 'ROPI5', 'NEOSYRD12.5W', 'VALDIND', 'GLAR100I', 'GAST30L', 'MIDA50I', 'VANCOINV', 'CORTIND', 'RALT400IND', 'VANC500L', 'LANT750', 'METO25', 'BACTDS', 'DIAZE5I', 'ACET100D', 'MORP20/30L', 'MAGSUL20I', 'LITH450SA', 'DOXA1', 'CORTOTIC', 'ENOX60I', 'PROP100IG', 'FENT100P', 'LEUC5', 'VANC.000125L', 'MYCO2C', 'DISU250', 'FLUC60L', 'ACET800I', 'SNLR260O', 'CLAR500SYR', 'DICL25', 'NS20SYR', 'HYDR1O', 'CHLO2', 'SORA200', 'VITE100', 'COMBPTCH', 'NTG100PM', 'OXYB5', 'CYCLOBASE', 'GOLY4L', 'ISMO10', 'MICO745C', 'DRON400', 'LATA5OS', 'FOSA150IV', 'DARU400', 'LAMO100BRAND', 'NIFR150', 'FENT20I', 'GLUC5XL', 'ENOX120I', 'TEMO100', 'CISA10I', 'HYDZ25', 'FURO40', 'LIDO100S', 'GLYCPED12', 'IRON25I', 'BACT10I', 'HYDR12.5LF', 'HALD5I', 'HALO10', 'TELA250I', 'ZONI10L', 'DOXA4', 'PHEN65I', 'SEND6SYR', 'NEOAQUA', 'TOBR3ES', 'TRDI40I', 'IRON100I', 'TRETL', 'CLINBASE', 'F

In [4]:
theirs = ['DXIG40I', 'NORESTFE', 'ARIP10', 'BACI5I', 'CALCITVITD', 'D12.5W250I', 'NEOLOPE2L', 'METR500I', 'LACO50L', 'OLME20', 'SILD25', 'MYCO250IND', 'ROPI5', 'NEOSYRD12.5W', 'VALDIND', 'GLAR100I', 'GAST30L', 'MIDA50I', 'VANCOINV', 'CORTIND', 'RALT400IND', 'VANC500L', 'LANT750', 'METO25', 'BACTDS', 'DIAZE5I', 'ACET100D', 'MORP20/30L', 'MAGSUL20I', 'LITH450SA', 'DOXA1', 'CORTOTIC', 'ENOX60I', 'PROP100IG', 'FENT100P', 'LEUC5', 'VANC.000125L', 'MYCO2C', 'DISU250', 'FLUC60L', 'ACET800I', 'SNLR260O', 'CLAR500SYR', 'DICL25', 'NS20SYR', 'HYDR1O', 'CHLO2', 'SORA200', 'VITE100', 'COMBPTCH', 'NTG100PM', 'OXYB5', 'CYCLOBASE', 'GOLY4L', 'ISMO10', 'MICO745C', 'DRON400', 'LATA5OS', 'FOSA150IV', 'DARU400', 'LAMO100BRAND', 'NIFR150', 'FENT20I', 'GLUC5XL', 'ENOX120I', 'TEMO100', 'CISA10I', 'HYDZ25', 'FURO40', 'LIDO100S', 'GLYCPED12', 'IRON25I', 'BACT10I', 'HYDR12.5LF', 'HALD5I', 'HALO10', 'TELA250I', 'ZONI10L', 'DOXA4', 'PHEN65I', 'SEND6SYR', 'NEOAQUA', 'TOBR3ES', 'TRDI40I', 'IRON100I', 'TRETL', 'CLINBASE', 'FMLSUSP', 'NYSTC', 'RETR200I', 'THIO100', 'DIVA125', 'MAFE8.5C', 'THOR25I', 'PRAZ600', 'COMP10I', 'LOVA20', 'INTR20/500I', 'MILR5I', 'KAOP30', 'ASA81', 'METL5', 'COZAA50', 'DICL50', 'FOLI1', 'KAYX60', 'PHEN15', 'CAPT50', 'DEFE500', 'OLAN7.5', 'MANN25IV', 'PHEN100', 'APAP500', 'CYCLOP10L', 'NATTTS', 'MAG20/500', 'AMIC250L', 'DESMIND2', 'VITBC1', 'HYD25PCA', 'ONDA4', 'PRISMAK5000', 'MAAL30BTL', 'PROM25R', 'CAND16', 'BETAVAL.1/15O', 'TEMO20', 'TILAIND', 'OSEL12L', 'CVVH3556I', 'ACYC400', 'IMAT200L', 'LEVO112', 'BETA30I', 'ABAC20S', 'TERB25', 'AUGM875', 'ACY500I', 'CALC500', 'RIVA10', 'LORA20I', 'MARIIND', 'GLYCO4I', 'EDRO10I', 'CLOB.05C30', 'BUPR75', 'IBUT1I', 'THIA50', 'VANCO10I', 'EPLE50', 'PAPA30I', 'CALC400S', 'NTG2O', 'NEOKCL', 'BETAH1', 'OXYT20/1000', 'GENT10I', 'VANC500I', 'OLAN2.5', 'THOR10', 'OCTR50I', 'FOSC24I', 'LIDO1/10I', 'CAPS25C', 'OXYCO5L', 'PARO10L', 'CODE30I', 'DEPO200I', 'MAFE5', 'FAMO20I', 'BETH5', 'MILR10I', 'MEGE400L', 'COLC0.3HT', 'NEOCARB200L', 'FURO80', 'BUME.5I', 'ROSI8', 'TRIVIT', 'INSNEO', 'HALO5', 'OXCA600', 'DARV65', 'SWSYR20', 'LEVE500', 'D54.25T1LI', 'DDI25', 'PRAS10', 'BUME05', 'APRE125', 'ESOM20', 'HEPA30I', 'IMIP50', 'SYRINGE60NS', 'VALS160', 'IBAN3I', 'PROG8G', 'NPHEN10I', '5FU500I', 'PHYT10I', 'PRISMACA5000', 'SENOKOTS', 'CIPRO400I', 'GLIM4', 'H2B1ED', 'OSEL6L', 'MAGS1I', 'NEOSYRD7.5W', 'METH125I', 'FOSI10', 'AUGMSUSP', 'CHAR120L', 'VERA120SR', 'ENOX80I', 'ADAL30', 'CHLO250', 'DDAV400I', 'MORP1-4', 'CADDNS250B', 'DIDA250', 'RITU500I', 'ESTR125', 'IPRAPF', 'FLUV20', 'GEOD80', 'TRIM200R', 'UNASNEO', 'PHEN250I', 'PHYT2L', 'FLEC100', 'BUPI0.1CADD', 'MTX50I', 'ENO3I', 'DESMOIND', 'ANTI250I', 'HYDR3R', 'INDO75XR', 'D25W250I', 'MEDR10', 'CYCL25', 'NEOCAL', 'VERA180SR', 'FERU510I', 'PROM25I', 'FLEETEN', 'FENT5I', 'CLOT130C', 'MEPH100', 'TACR5SM', 'LITHI300L', 'FTY7202.5IND', 'PERI', 'OCTR500I', 'LEVE250T', 'DOXY100C', 'OXYC40', 'DIAZ2', 'NYST60LSM', 'MILKTH350I', 'NALB10I', 'BETA10', 'FLUD100', 'HEMOSUPP', 'DEXA1ES', 'ALTE2I', 'NATE60', 'QUET300XR', 'H1B1ED', 'LACO50', 'MESADESEN10', 'TEGR200', 'MAGN50I', 'VINC2SYR', 'ISOS2.5SL', 'PRED5SM', 'VALG450SM', 'IMDU60', 'INSULINCARB', 'PANC16', 'NEPH2.71000I', 'MORP20/120L', 'METZ50', 'PANC10I', 'COSO10O', 'OXAL100I', 'FENT100T', 'PRAM125', 'PANT40I', 'TEMO5', 'GLYC1I', 'IFEX1I', 'BIAX250', 'ACET1000PB', 'CATA1', 'KAYEX30R', 'CLIN1S', 'SEVE800', 'TRET.05C', 'LIDO2000I', 'PYRI10/100L', 'ISMO20', 'BUPNAL82', 'THIAIND2', 'BACT2/1O', 'RIVA2L', 'LIOT50', 'NORE1600SYR', 'NORE800SYR', 'PYRI180', 'MILR20PM', 'DEPA500ER', 'DICL500', 'METH12I', 'BISO5', 'DOFE250', 'NEOVIDAYLIN', 'SARG500I', 'AZAT37.5HT', 'LEVO25', 'TORS20I', 'VANC12.5I', 'GUAI600T', 'COLEPWD', 'GUAIC120L', 'HCTZ25', 'TOLT1', 'POTA10PM', 'CLOPIND', 'PROP150', 'MESADESEN11', 'FTY7201.25IND', 'ALLO20S', 'POLIOV', 'DESM0.1', 'GENT3EO', 'OXYM0.25', 'ROPI0.2CADDB', 'PROC5', 'MAGO140', 'PROP20L', 'LORA1', 'IRBE150', 'VZIG625I', 'ARIP9.75I', 'DEGA120I', 'BARI2/450L', 'AMPR150', 'TRET40L', 'HUMU100I', 'PILO5', 'ROPI2', 'ANUSOL', 'AMOX500L', 'OLSA250', 'EPOP5I', 'SULF500', 'AUGM1.25L', 'RITA10', 'FLUM500I', 'DESI50', 'MORP2IV', 'MITO5I', 'DAKINHS', 'NEOLORAZL', 'MIDO5', 'EFAV200', 'NEOFLUCIV', 'HEPA10SYRN', 'METC750', 'INSULIN', 'ASCO500', 'SIRO6L', 'OXAC150', 'THEO100', 'ALTE1I', 'METO10PB', 'CALC.5', 'KCL20PM', 'ORBE5T', 'CEPHDESEN1/10L', 'CEFT2F', 'HCTZ50', 'MARA300', 'PERA200I', 'CYCL1I', 'NTG15P', 'MINX25', 'H1H1VAC5I', 'FLUT1C', 'DAUNO20I', 'RISP1', 'CANGIND', 'HALO05', 'METH1', 'RIFA550', 'NAVA250I', 'HALO50I', 'CSA100L', 'MPRED16', 'CHOL500', 'SIMV10', 'LEVO1.5', 'MESNA1I', 'LABE100', 'HER440I', 'SELE40I', 'CISA20I', 'MORP4-10', 'NEOMOR5', 'SOLI5', 'ALBU25IVPB', 'DOXE2I', 'LEVO150', 'TOLA250', 'PERM5T', 'MORP2-8', 'KETO2SH', 'CLOB5', 'LIDO1BUFF', 'SIRO1L', 'PHEN130I', 'LANS30L', 'RIBA200', 'HYDR2I', 'OSEL75', 'ACI25', 'LANS30ST', 'FLUC250', 'ENBR25I', 'TBDX2.5O', 'MYCO1500L', 'PREV60TP', 'PREG5L', 'TEST25P', 'MICROK10', 'LANS30', 'DIPH2.5L', 'CACHL30L', 'VANC5I', 'PHEN10', 'LEV500', 'TOL1C', 'BORT3.5I', 'CHLO10', 'MESA5S', 'NEOERGO20L', 'AMBI5', 'FLUCO400PM', 'MYCO150', 'PIRO20', 'LEVO5ES', 'CIPR500', 'LORA2PB', 'BASIS', 'DIVA250', 'IBUP600', 'MEPE500PCA', 'LOMO', 'CODE30', 'HEPA5I', 'TALC', 'ETHA25', 'FENO160', 'TRIA.1/80C', 'DOXE50', 'ASCOR504L', 'RILU50', 'CYAN100', 'METHB1I', 'FLUT220H', 'LIDX60O', 'LIDX15C', 'TAXO160I', 'NALO4ILF', 'DARU600IND', 'NIMO30', 'CEFI100L', 'EPO20I', 'DICL75', 'ZILE600CR', 'PYRI50', 'PENI250', 'PHYT4L', 'THIO500K', 'DARU600', 'GLYB25', 'TREP20I', 'QUET100', 'PB15', 'CEPH50L', 'CIPR750', 'PARO12.5CR', 'VANC125L', 'MOVIKIT', 'CEFR250I', 'RIBAV6I', 'HEPVAC', 'ERGONEO', 'AMAN100L', 'AZTR1000I', 'ASPAE10000I', 'FLUP1', 'CLOB005O', 'METF750XR', 'CELE100', 'FEBU40', 'MESADESEN8', 'PIRO10', 'HYDR20/100NS', 'PEGA75I', 'AMOX125L', 'COLL30O', 'TERC80V', 'ARIP25HT', 'AMIL5', 'LEFL10', 'ZIDO10L', 'DEXA100I', 'THYR30', 'PRAM.25', 'LAMO100BN', 'ADDE5XR', 'DOXE10', 'GLUCTAB', 'OLAN10I', 'VINB10I', 'AUGM250', 'PIND5', 'TROS20', 'PROP200IG', 'IPRA2H', 'TACR0.5', 'INTR10/250I', 'BACTDES3', 'TERC.4C', 'MCT4L', 'SNLR160T', 'LIDEPF1.530I', 'TACR0.1O', 'EPOP1.5I', 'CLON5', 'TRIA0.1/80O', 'ZOFR4', 'ROSI2', 'DOLA100I', 'NADO20', 'PALI100I', 'FLUM1I', 'EPO2INESRD', 'RITU100I', 'DASA70', 'EFAV600', 'AZAT25HT', 'FELO2.5', 'PROZ20L', 'CACL1000I', 'MEPE25I', 'PERP2', 'TACR0.5SM', 'ACET0.04IR', 'NOREBASE', 'ARGA250I', 'OCTR30I', 'BACTDES6', 'AMPI500L', 'LOPE1L', 'NHYDRA', 'PRIM', 'CYAN1I', 'ERGT1R', 'SIRO5L', 'DAKINES', 'FURO40I', 'ANUS1O', 'HEPA5000I', 'DAKINSF', 'CYAN1000', 'TENEIND', 'STAV40', 'ALBU4R', 'GLIP5', 'PYRI25', 'CVVH', 'TERAZ5', 'SODCIT3SYR', 'MSCO30', 'ETRA100IND', 'CAFF250I', 'GLYB1.25', 'GLUC1I', 'ENAL10', 'GLAT20I', 'AZTR2I', 'MESADESEN1', 'METF850', 'PHEN5N', 'DRON10', 'VECU10I', 'DOXA8T', 'PILO5ES', 'MISO25', 'CYTA50I', 'DIGO125L', 'PHOS250', 'CHTH25', 'VENL375XR', 'DIG5P', 'CITR30L', 'METH10SY', 'IMAT100', 'RIMA100', 'OLAN10', 'FLUN1H', 'TRIF5', 'SIRO8L', 'AMPI500', 'ALPR25', 'FAMC500', 'FORT200', 'LACO200', 'LOPE1/7.5L', 'ATRO4I', 'DILT90SR', 'PROP200I', 'TRAN1', 'MESADESEN5', 'CADDNS250', 'NATH25I', 'CEFANEO', 'MEXI200', 'PILO2ES', 'SYNE500I', 'ISOS40ER', 'MENI1SYR', 'VITA50000I', 'ZIPR40', 'THIX5', 'CYCL1/2ES', 'STLTP', 'ALTE2IV', 'KETA100SYR', 'LEV250', 'SODCIT$/500I', 'TBSYR', 'AMPI250L', 'RAMI5', 'BACTSSSM', 'NEOACET2', 'CYAN500', 'ALTE1/5I', 'PENDESEN', 'FISH1000', 'STARTNEOD5', 'TERLIND', 'NICA30', 'MAG4PM', 'CYTA2000SYR', 'ASA300R', 'NEUT1I', 'LOPE2', 'LEVE100L', 'FENO145', 'INTE9I', 'SLO300', 'HALOPB', 'DIGO125', 'VANC1I', 'PANT40SM', 'RAMI2.5', 'EXCELNS1000', 'FLUP5', 'EPO3INESRD', 'nmetroni', 'FLUP2.5I', 'PIOG45', 'VERA5I', 'PERCBULKL', 'APRE80', 'ATOR10', 'TIOT', 'OXYM15', 'MORP1-8', 'LEVO100I', 'NS500IND', 'TOLVIND', 'RITU100IND', 'PRED125ES', 'CLOZ25', 'PAMI90I', 'LURA40', 'HOMA2ES', 'FOSA10', 'FINA5', 'NEOACET', 'RIBA1I', 'OXAC1I', 'IRBE300', 'FURO40L', 'LEVO75', 'PRAZ2', 'NITP50I', 'ATEN25', 'DOCU100L', 'LEVE500I', 'CLOZ100', 'LEUP75I', 'ETOP100I', 'DIDA200EC', 'KCL40I', 'CSA100', 'MAXISES', 'BACT2/15C', 'HEPA10MUI', 'NIFXL90', 'BETA0.05L', 'CEFA1000I', 'ENTE1', 'CLO025L', 'SIRO1', 'KETA5I', 'FAMO20PM', 'ISOS40SA', 'MYC30O', 'MPRED5I', 'BAYHEP.5I', 'MELO7.5', 'SULI200', 'AMIO150I', 'SEVE403', 'VITA25', 'TRIM250', 'BENZ1', 'FLUP25', 'PARO20', 'NS10I', 'NEOEOP4I', 'CILOS100', 'FOSP100I', 'VITE1000', 'CAPT125', 'DEXA4', 'AMPI250', 'NITR25', 'THIO25', 'METH1000I', 'LACT12T', 'RENV800', 'MEGE40', 'LEVS125', 'HYDR10', 'ATGDESEN', 'CANC50I', 'BENZ2.5G', 'NORT25', 'PHYT5', 'ERYT2S', 'LEFL20', 'DILT60', 'ACETSP', 'MAXIEO', 'DEFE500IND', 'NITR100', 'NE1/35T', 'OXA2I', 'OXAZ10', 'GLYC1', 'ROPI1', 'METR1G', 'ATRO1I', 'VANC500PR', 'IVIGBAG', 'VERA240SR', 'SILV1T', 'KBL120L', 'CLOFPLACIND', 'HEGB1I', 'DROTIND', 'DIPY25', 'BAG100', 'SUMA6V', 'EPI1N', 'INFL100I', 'MSCO100', 'AUGM875L', 'EPHE50I', 'TOPR50', 'NORMOCARB', 'NOVONPH100I', 'TTOX5I', 'DOMEPKT', 'ANID100I', 'CYTA100I', 'OCTA10I', 'CHO1I', 'INVE3', 'ZIPR20I', 'MSCO15', 'OLME10', 'MORP1-10', 'FLUV100', 'AMIN500', 'PRAV10', 'ETHA50I', 'AMID100HT', 'CHPR250', 'GLU125/250', 'LEVO200', 'PARI5I', 'IMIG400I', 'ACET650R', 'UREA10L', 'NFAT5', 'NALO1I', 'MPRED40I', 'ARIP15', 'SYR60ML', 'INRPI', 'ACET40', 'INDO25', 'DEXA75', 'MAXI1ES', 'DISO150CR', 'AMAN100', 'ALPRNEO', 'CALCIT950', 'TACR5', 'PHEN30', 'ENAL25IVPB', 'TOPI200', 'APAP80R', 'GEMT5I', 'FOND2.5I', 'ATOM60', 'DAPS100', 'ESMO2PM', 'TORS20', 'LEV100', 'CEFE1000I', 'MESN1I', 'DIAZ10', 'VENL75', 'ACET325', 'FLUT16N', 'NEOPOLYVI', 'DESO15C', 'NS100H', 'BUP01ED', 'PRAM..5', 'CALC1I', 'DEXA15', 'D7.5W250I', 'FLUC100', 'ASAB325', 'CLORA7.5', 'MERO500I', 'PROP05ES', 'PHENO100', 'METE2I', 'NS100CG', 'ACD3/1000I', 'NFAT6', 'NEOCEPH', 'ESCI10', 'PROP60', 'KAYX30', 'DIVA1750L', 'CALC100SI', 'TIZA2', 'CYAN250', 'BETA0.05O', 'NEOMETO', 'MPREDDESEN', 'PROP120', 'QUET200', 'MTX200I', 'SULI150', 'RISP25I', 'DIPH125L', 'AMOX50', 'NOVO100I', 'LEVOBASE', 'ETID200', 'SIME80', 'OXAC1F', 'FENO48', 'PAMI30I', 'DROP5I', 'ESMOB100', 'DROT20I', 'ESTR0.025T', 'VERA80', 'QUIN10', 'BAG3000mL', 'NITR20SYR', 'WARF1', 'NAAC20I', 'ENTE0.5', 'PROP40', '5000MLBAG', 'SECR16I', 'RISP05', 'CYCL50', 'CYAN50', 'SALM65H', 'HBIG5I', 'SCOP.25ES', 'THRO10T', 'SULF10EO', 'GLIM1', 'SIRO3L', 'NEOCAP', 'LOXA10', 'SODCITABL', 'ASCO500I', 'PHEN50', 'MCT30', 'LIQUIDE', 'PRED50', 'ROMI250I', 'NAPH1O', 'GANC250C', 'HEPA1000/500', 'LIDX15O', 'TOLV30', 'METR250', 'NITR50I', 'TRIA0.1O', 'ZAFI20', 'SODI4I', 'ATR1EO', 'DEFI80IND', 'NACL234', 'TRIA10I', 'ILOP2', 'BISA10R', 'BECL16.8H', 'NEOSES', 'FAMO20SM', 'LEPR50I', 'TERB1I', 'TREP1.74', 'ZON100', 'INSU100SYR', 'TOBR300H', 'ROPI25', 'NEOSPIR', 'AZIT600', 'DARB200I', 'SEVE400', 'DOXO50I', 'NEOSPIR4I', 'ESTR25I', 'THOR100', 'IMDU30', 'DAU50I', 'DEXR500I', 'RUFI200', 'MIRT30', 'NADO40', 'CLOP05L', 'VIAL', 'EPI1I', 'GLIM2', 'NICA2.5I', 'LIDO5O3.5', 'BENZ10O', 'MEPI1I', 'CYTO5T', 'DEX50SY', 'ARTI3.5O', 'CLIN300PM', 'ERYT250L', 'INDCLOF20I', 'MORP4-8', 'OLAN10ODT', 'FEXO60', 'NIFXL30', 'NEOLEVO', 'OXACDESEN', 'RISE35', 'EPO4INESRD', 'CISP50IV', 'HYDR1I', 'FENT250I', 'ALEN70', 'MITX20I', 'COMBTAB', 'SIRO0.5', 'METH40IVPB', 'MPREDSYR', 'META800', 'PEPC20I', 'CALC667', 'HYDRNEO1L', 'PENI1F', 'CEFT500I', 'RIVA15', 'PHEN10I', 'THEO200', 'METR50L', 'EZET10', 'LIDX60C', 'METOSUSP10L', 'THEO80L', 'CHOL750', 'PROZ20', 'APRA5OS', 'TUBE5I', 'ROPI0.2/100I', 'TPN', 'DAKINQS', 'CARB200XR', 'TOLC100', 'INTEGR', 'CHL500PB', 'DERMGEL', 'TOLT2', 'LR5SYR', 'CYCL1IV', 'TIC250', 'METH36', 'ACE250', 'NEOMID2.5', 'LITHI60L', 'KET15C', 'RISP025', 'LIDO1I', 'DERMON', 'CHPR100', 'CEFA5ML', 'PYRI60L', 'DROP5PB', 'AZAT75', 'NEOAUGM50S', 'HEPA10S', 'RITO80L', 'LEVO250PM', 'KPHO45I', 'VALG50L', 'DESI100', 'LEUP22.5I', 'RISP37.5', 'DABI150', 'FOSA150I', 'MBLHCV1IND', 'DILT125I', 'INTE10I', 'HALO1', 'PROM25', 'ERYT25L', 'ERGT1', 'ACET650L', 'METH40', 'QUIN800I', 'DOXE0.5', 'WARF2', 'NIFE10', 'MIDA5050', 'PHENY100L', 'QUET12.5HT', 'ITRA100', 'MAXZ1', 'NITA500', 'BETAQL', 'LEVE500BN', 'NICO7P', 'CHLO4', 'IODO210', 'NTPN', 'NAMP50N', 'HALD100I', 'MESA1000R', 'NEOR100', 'CLIN600PM', 'LOXA25', 'MORP1000I', 'CYCL100I', 'LORA10', 'MORP5I', 'PGBNEO6I', 'IBUP200L', 'HYAL200I', 'CAL1250', 'DICL250', 'ATAZ200', 'ZOSYNDESEN', 'ASA81EC', 'RIS1L', 'LINE600', 'NEOFER', 'MEGE20', 'MORPIM', 'CLON2P', 'LIDO/EPI30', 'EMLA2.5', 'NEOR25IND', 'CARB25', 'PB60I', 'THRO1T', 'PENT10I', 'NEOSEO', 'HYDR1000L', 'FAMOT20I', 'CEFU75I', 'HYDR500L', 'CITA20', 'BUME25I', 'NACL1', 'PANT40IVPB', 'FINA1L', 'PYRI100I', 'VENL375', 'RIFANEO60L', 'NEOR100IND', 'SPIR25', 'ZITHR250', 'PRIV5I', 'FERR62.5I', 'DICL10', 'INDO50', 'DISO100', 'HOMA5ES', 'NEOKCL1.3L', 'MOXI400', 'IMAT400', 'HYDZ20I', 'HC5', 'PILO1ES', 'THAL50', 'IVIGBTL', 'PHYT10IVPB', 'QUIG324', 'BAG', 'ENAL25I', 'LAMI10L', 'CEFTR1000I', 'ERGO400T', 'KAYE15L', 'GENT100PM', 'BUSP10', 'IBUP400', 'LIDO5T', 'PENT100C', 'DESI30O', 'DEFI80I', 'UREA40C', 'PEME500I', 'NS1000IBTL', 'NEOSYRD5W', 'MYCO250L', 'TORS100', 'AMIK500', 'LEVO88', 'KBL240L', 'RISP3', 'IMAT400L', 'DOXE25', 'CABE0.5', 'CEPH250', 'SILD50', 'DOFE125', 'CLOT10T', 'DIDA125', 'ARSE10I', 'DAKINHS480L', 'VITAMA10000', 'BACTSS', 'SAQUIN200', 'PROC250', 'DARA25', 'TIGE50I', 'THYR120', 'HYD12.5PCA', 'HYD2530C', 'DIVA125SP', 'METH20I', 'PAPA150', 'ANAG500', 'CARB200L', 'HYOS.375', 'NABU750', 'CARV31.25L', 'BENZ2', 'DOXO20I', 'ACE500I', 'EPO3I', 'EPO10I', 'ERGO50', 'PHYS2I', 'GENT3ES', 'FOSP500I', 'NEOEPO', 'CLOP15L', 'LEVO200I', 'ALTE1/2I', 'DIVA250ER', 'BUPR100', 'PEXEIND', 'CLIN9000I', 'AMOX875L', 'DIPI1ES', 'PROT50I', 'NEOSYRSW', 'MORP10R', 'NTG65', 'ASCO250', 'SYRINGENS', 'THYM25IND', 'CAPT25', 'DIDA400', 'MICO2P', 'BIVA250I', 'REFRESH', 'NEOPHOS2', 'CETX2I', 'HCL1MEQ', 'NEOURS', 'TRAZ100', 'FLUCDESEN', 'SW10I', 'FLUO2500IVP', 'LEVOBASE2', 'TRAZ50', 'METR1000L', 'BERA8I', 'PHYT3L', 'THOR25', 'TROP1ES', 'ENTE0.05L', 'METH40I', 'TRIA25', 'FURO10L', 'SIRO7L', 'AMLO5', 'D15W250I', 'FEN625PCA', 'ASPI81INDP', 'HC100IC', 'COMB14.7H', 'NICA40PM', 'INNHI', 'NTG4SL', 'AMP50I', 'VENF25', 'MORP2-4', 'D20W500I', 'SIME40L', 'DIBU', 'VARI125I', 'CARM100I', 'HYDRSYR', 'DONE10', 'EXEM25', 'NFURON', 'NEONYST5L', 'HYDX50', 'NABC50IV', 'TIRO12.5/250', 'PHEN100IVPB', 'ADVA250IH', 'HYDR2', 'CANC70I', 'TACR1SM', 'LANS15', 'METH50L', 'HEPPREMIX', 'ETHA100', 'CLOT1VC', 'COLO06S', 'NAAC100I', 'RASB1.5I', 'PRED1', 'CAPT1', 'CEPHDESEN50L', 'CEFX1F', 'LIDO2/5J', 'PRAV20', 'ETOD200', 'GABA250L', 'CLOR375', 'LID2J', 'TIAZ300', 'INTE15I', 'MORP250PM', 'ROSU5', 'CLIN600I', 'FOLI1I', 'METL25', 'ZIDO100', 'STAV30', 'CLIN150', 'NORMOCARB2', 'METO10L', 'AMIN100', 'TENO300', 'VITE400', 'POLY15O', 'NEOSYRNS', 'DURA', 'ACET1000I', 'BLISTEX', 'NORT10L', 'PYRA500', 'CLOB0.05S', 'FENT2.5I', 'CARV125', 'LAMO100', 'DYAZ1', 'MIRT15SOL', 'MOLI5', 'TETRA5ES', 'METYR250', 'BACL10', 'SEPKET10I', 'MICA100I', 'NEOS0.5I', 'BUPR100SR', 'SUCC20I', 'TOPR100', 'TACR1', 'SCOP15P', 'TRIAM.025/15C', 'CATA2', 'MESADESEN4', 'QUET50', 'CLOP75', 'MSCO60', 'LIPO20200', 'STARTPND10', 'ERYT500I', 'VAP500I', 'AFRIN3ML', 'SINE25100', 'CLOP1L', 'DIAZ10I', 'ACYC800', 'COLC5', 'RANI150L', 'MEPE100I', 'ROSU20', 'TRIM100', 'NORT10', 'PANCNEO', 'BENZ5', 'SINC50200', 'DONN162', 'TDAP0.5I', 'MORPCONC', 'DEFE500I', 'DOPA400PM', 'CINA30', 'SELE5', 'ROFE3I', 'DTAP', 'PRED25', 'RABI2I', 'CHLO500', 'TORS5', 'BENA5', 'OXA10I', 'MINO100', 'DOPA400D', 'FELOD5', 'MONT10', 'DOLAS100', 'PERP4', 'GLU25/500', 'ABAT250I', 'GAMM10I', 'ENFU90I', 'SUNI25', 'LUBI24', 'THYR180', 'PNEU5I', 'PIMO2', 'BTL250', 'VANC12.5L', 'FERR300L', 'AMPH250I', 'GENT80PM', 'TENO300L', 'D10W250', 'GANCIND', 'ALEM30I', 'EPI30H', 'LAMO100XR', 'SOLN2', 'CIDO0.25S', 'FLUO500I', 'SW1000', 'BORT3.5SC', 'BENZ100', 'GUAN1', 'CEFDES', 'NORT50', 'DOXY100I', 'EPIBAG', 'REPA2', 'CIPR200I', 'PRAV10L', 'FLUC400L', 'TEST5P', 'TRIF1', 'CYCL2O', 'CRIT6T', 'FLUC200SYR', 'LAMI150', 'TIMO25ES', 'PARO10', 'RALO60', 'CYPR2L', 'ZILE600', 'SYR1ML', 'TIAG4', 'NEOR100L', 'NEOPROP4', 'INHRIV', 'DILT90', 'VORI50', 'NEOEPO10I0', 'ERLO150', 'QUIN40', 'DISO100CR', 'CLOP2.5L', 'THOR50', 'TERB1SP', 'MISO100', 'ERY1I', 'FLUC200SM', 'MONI7C', 'TIAZ120', 'FLEC50', 'HALD50I', 'MORPITPACU', 'AUGM500', 'COLE625', 'METH5', 'CIDO1P', 'NCEFXIV', 'SW5SYR', 'NEOSYRD5W50', 'PROP50', 'TOBR1200I', 'ERGO8000LNEO', 'CIPR50L', 'VALC60L', 'MEPE100AMP', 'CEFP200', 'MISO200', 'PERC', 'VANC250L', 'FOND5I', 'CHLO500L', 'NEVI200', 'INLHI', 'ASTR5I', 'NARA2.5', 'HYDM4', 'NEOZOSYN', 'RIFA600I', 'PROC375', 'DILA30', 'MEXI250', 'ATOR20', 'VITA5I', 'CLON1I', 'CEFX2F', 'NAPH15O', 'MIRT15', 'FENT12P', 'ETHO250', 'ATROP1I', 'TIAZ360', 'GEM1I', 'EPI30I', 'ANHC25R', 'METH20SR', 'SUNOLIND', 'DAPI0.5ES', 'OCTR100I', 'LORA5L', 'BECA15G', 'PROC5L', 'CLOP300', 'AMPH500I', 'MORP50PCA', 'PRAM0.5', 'NEOZDV', 'GOLY60L', 'CLOP0.5L', 'FOND7.5', 'GLUL100I', 'CHLOR5', 'DAPS25', 'HYOS.125', 'LACT500R', 'PEDDIG0.1', 'RISP2', 'PALI6', 'CLIN900PM', 'NEOM30O', 'SIMU20I', 'DIPH25', 'METH8', 'ITRA10L', 'METHCONC', 'DELA100', 'ACYCIND', 'GEMF600', 'BENA20', 'BETAVAL.1/45C', 'NEOSYRSL', 'POSA40L', 'DILT500I', 'TAXO80I', 'TERAZ1', 'SILV1C', 'CELE200', 'VANC.0125L', 'PALINEO', 'ALTE50I', 'PRAZ1', 'NEUT', 'LITH300C', 'LEUC25', 'GENT20IVT', 'FLU200PM', 'BETA10O', 'IMIP500I', 'ADEN6I', 'DANA200', 'LANT500', 'MECL25', 'CILL50', 'ONDA4I', 'MORP20R', 'KASE40I', 'DT.5I', 'TORS10', 'PYRI50/25L', 'LANT250', 'BISA5', 'IBU5S', 'CEPHDESEN1L', 'CEFT2I', 'HYDRO4I', 'LAMO25', 'LIOT25', 'RASA0.5', 'LIPIDSTUDYP', 'MESADESEN2', 'AQUA400O', 'NTG100PB', 'MOEX15', 'TRIF1O', 'HYDR1C', 'AGGRCAP', 'EPIPEN0.3I', 'ZONI25', 'NS1000OXYT', 'WARF5I', 'HYDZ50', 'TERI750I', 'EPI400SYR', 'ESTR625', 'CALC.005C', 'NIAC100', 'MORP30', 'KETO200', 'NEOSI', 'MOR10L', 'MARA150', 'CROM40N', 'IMIP10', 'GLUT5', 'CALC13.6I', 'ALEM30SYR', 'THEOP125', 'ONDAN4I', 'DARU300', 'PROP1I', 'PYRI60SYR', 'THIA100', 'FOSC12000I', 'RANOL500', 'CARB100T', 'CLAR250L', 'RALT400', 'ALLO500I', 'NEOLEVO5', 'DIAZ50L', 'METH2I', 'DACL25I', 'CARV3125', 'PRED10', 'LIND1S', 'ADAL40I', 'HUM100I', 'BAG250I', 'MYCO500T', 'CAND4', 'METO50', 'MVITH', 'PROM125', 'ETHA400', 'NCEFZIV', 'DIPY75', 'GANC500', 'CEVI30', 'MIDA2I', 'THYM25I', 'NTG3SL', 'RIFA200', 'NEOSYRD10W', 'METP250', 'LOXA5', 'METO100', 'AUGM50L', 'AMBI50I', 'TRMP100', 'PEPTO30', 'NALO4I', 'TERB5', 'BACI15O', 'VORI200I', 'ACET.125T', 'AMP500I', 'PEPC20', 'DISO150', 'NEOCHLO', 'PCNGB600I', 'MPRED80I', 'RIBO50', 'AUGM250L', 'AMIT25', 'MGBASE500I', 'DILT25I', 'NACL4I', 'RISP50I', 'RANI150I', 'QUIN5', 'canc50i', 'LABE200', 'SINC5I', 'DIVA500', 'INSNEO1', 'MINO100I', 'EPO20INESRD', 'LACO200I', 'RHIIND', 'ATOR80', 'VINC2I', 'FENT400L', 'BETAAUG0.05G', 'MACR100', 'FENO10I', 'BLEP2ES', 'OXALDESEN', 'LEVO4I', 'D5W', 'HC500I', 'VITD1000', 'PHEN12SYR', 'PROP1PB', 'FURO20I', 'EPIR50I', 'HYDU500', 'FLUO20L', 'SODI3N', 'NOVOR100I', 'KETA200I', 'OMEP40', 'NEWMED', 'EPOHD', 'ASPARSYR', 'ZINC220', 'PYRI10I', 'RISP1L', 'KPHO150I', 'URSO250', 'ENOX150I', 'SALM13IH', 'PRISMASOLK32', 'OXAZ15', 'CETU100I', 'LEVO125', 'CEPH500L', 'RITA5', 'FENT25P', 'SINM25100', 'DEXT500B', 'DOXO50SYR', 'ILOP6', 'PROAIR8.5H', 'LEUP45I', 'ATGAM', 'ATRO1ES', 'CALGLUCNEO', 'FULV250IND', 'VANCO25O', 'POSA200L', 'PROG100', 'ACCUSPAY', 'NEOTRIVIT', 'TEGA2', 'DESI2C', 'WARF25', 'BETH25', 'FENT500I', 'ZDVLIQ', 'PHEN50I', 'CHLO180S', 'HYDZ10', 'LAMI250', 'BUTO1N', 'POLYB1MUL', 'METAMOR', 'THIA100I', 'PERP8L', 'BUDE3', 'COLC6', 'UROC10', 'PERG25', 'METF500', 'TOPI25', 'HYDR1AMP', 'HYDRO100I', 'SULFA500', 'ACY5O', 'LEVOX125', 'INTEG20I', 'VALS80', 'PROC750SR', 'ASPI325INDP', 'MYCO250', 'MIDA5I', 'AZA100I', 'BACT2C', 'HEP10I', 'COAL3S', 'BUPI0.05EPD', 'TACR5BN', 'POVI30O', 'HC100I', 'OMEP20S', 'DAUN20SYR', 'PRIM250', 'FOLI5I', 'TRIF2', 'ROFE25', 'MONT5', 'NEOINDOM', 'MORP25I', 'ZONI100', 'MEGE40L', 'ANAL15C', 'KCL10PM', 'BETAVAL.1/45O', 'NEOFLUC10L', 'IN73HI', 'ZEASP', 'VALP250', 'TOCA5O', 'BUPR2', 'LEVE1000L', 'CIME300', 'MEXI150', 'PRIM1I', 'PHEN25ES', 'BLEP10EO', 'ADAL60', 'TOBRFES', 'BETA0.05C', 'AMIT10', 'NIAC500SR', 'HEPA10000/5I', 'METL10', 'BACTDES1', 'OXAC300', 'ADDE20XR', 'PROC10I', 'VASOAES', 'BACI25/1L', 'CSA50I', 'DILT60SR', 'NAVEL10', 'SIMVIND', 'METR500', 'VANC.00125L', 'PROP500IG', 'OXYM1I', 'TOPI100', 'OKT35I', 'PERP5I', 'FINA1', 'SPIR100', 'MORP2-6', 'ANUSHC', 'SIRO1SM', 'FURO20', 'CEFU750F', 'CALC25', 'ATAZ150', 'FAMC250', 'CEFT1000DES', 'NYSTO', 'ACDA2.5I', 'DESI25', 'TRIAM.025/45C', 'GANC500I', 'CEFOT2I', 'LORA2I', 'HYDR0.5C', 'MORPH100', 'FULV125I', 'IDAR10I', 'MODA200', 'FLUP5L', 'DOFE500', 'ACYC200L', 'DULO30', 'HYAL150I', 'TERB1C', 'CARI350', 'TIPR250', 'NOXAI', 'HALO2', 'HYDRIND', 'DIDA100', 'NEOPRED2', 'RITO100', 'CITR6T', 'FLUR3ES', 'CEFO2F', 'TOBR3EO', 'SARG250I', 'MOEX7.5', 'HYDR500I', 'SERT100', 'PANT40', 'SARNL', 'BACTDES4', 'SALM50D', 'BUSU60I', 'ADEN90I', 'NPENI', 'MORP2000I', 'RIFA150', 'INUHI', 'REPA1', 'AUGM200L', 'MECL125', 'RISPM1', 'CART1ES', 'NITR30L', 'TRIF12.5I', 'DILHP50I', 'KAOP180L', 'OXAN25', 'NTG25P', 'GLU5/500', 'CALG500', 'SIMV40', 'NADO80', 'TRIM100I', 'CLON5I', 'PHYT1L', 'DANM25', 'PHENY100', 'NIAC500', 'MESADESEN7', 'FLUC200L', 'THIX1', 'CEFP100', 'MALA250/100', 'OXCA60L', 'PHEN10ES', 'CIPR0.3EO', 'PRAZ5', 'AZTR1000ID', 'SW3000I', 'DEXA1L', 'ALPR500I', 'NICO21P', 'RIFA300', 'PROC500SR', 'MESADESEN9', 'ALPH0.15O', 'DIGO25', 'NEPH1', 'HYD50PCA', 'AMANT50MG', 'ALBU%500', 'CALCG2/100NS', 'ILOPDOSEPACK', 'METO5I', 'DROT5I', 'BUME1', 'TRUV200/300', 'SODI65S', 'NITRO1O', 'FENT12.5P', 'MAALLIDO120L', 'DEXTROSE', 'DOXA5I', 'NTG100I', 'INSU500I', 'SW30SYR', 'ISOS20', 'NABC50S', 'ACYC1000I', 'GLYN3', 'THEO80LQ', 'LIPID250I', 'MEDR25', 'LEVO750', 'HYDR2.5ES', 'FULV250I', 'NESI1.5I', 'ANAS1', 'NICA20', 'NTG5P', 'ASCO1000I', 'QUIN325', 'PIOG15', 'URSO300', 'LORA2L', 'TEGA6', 'NEOHYDP', 'AMOX1000L', 'EPI15H', 'TAMO10', 'DEFE2000I', 'CYPR4', 'PILO6ES', 'SILD20', 'SYRINGE5', 'ESMO100I', 'POTA20', 'LOVE40I', 'DIAZ5I', 'LEUC100I', 'MPRED1I', 'METH500', 'FLUP25I', 'CYTO2500I', 'SUMA25', 'NERYIV', 'ESTR3', 'DIPH25L', 'NACL23.4I', 'MIDAZBASE', 'MESA4R', 'CLOP5L', 'DASA100', 'NSSYR10', 'AMP', 'MORP1-6', 'ATRO1/15ES', 'DEXA5', 'FLUC50', 'INDI200', 'METR500PM', 'ZIPR60', 'LOPER2L', 'ANUSHCOINT', 'ISOP1I', 'COLI150I', 'AMIO40L', 'BICA50', 'ACEB200', 'QUET200XR', 'KAOL60L,', 'UNAS15I', 'PYRI100', 'SODI650', 'TROP5ES', 'NICABAGNS', 'CLOB.05G30', 'AMIC5L', 'METRODESN', 'EMTR10L', 'CASP70DESEN', 'BETA10T', 'HC100R', 'LISI5', 'HYDX25', 'HC20', 'HYDR25I', 'LORA4L', 'LINE20L', 'TEST4P', 'CAPE500', 'FENTSOLN50', 'STRE1I', 'SITA100', 'TIAZ180', 'MICO3SUPP', 'EMTR200', 'TRET10C', 'MORP15I', 'ASA325', 'NEOLEUCL', 'TIMO5ES', 'SEND5SYR', 'PROC25R', 'TRIA1P', 'HYDR2PF', 'AMPOBORAL', 'ALEM10I', 'CETI10', 'GLUC2.5XL', 'POLY30O', 'THYR1', 'CARB200ER', 'ASPIDESEN', 'RITO100T', 'FOSI40', 'VALS40', 'HUMA75/25I', 'SULF100', 'EUCE120C', 'HIBTITER', 'FEN1250PCA', 'CLON3P', 'PRED0', 'ACAR50', 'FENT10I', 'COENIND', 'ROSI4', 'TELM20', 'DTOBULK', 'MTX25', 'ROSU10', 'WARF75', 'MICO2C', 'LORA1L', 'VOLU450L', 'ZOSY2000I', 'DDAV4I', 'CLOF20I', 'METH1L', 'F4B1ED', 'CLOB.05O30', 'CEPH500', 'FOSA700', 'RITU500DESEN', 'METH250I', 'MYCOC', 'NABU500', 'ONDA40I', 'DIPY50', 'TACRSUSPBN', 'NEOPREDL1', 'PSEUD30L', 'OCTR1000I', 'AMID200', 'D550.2ALB', 'ONDA4ODT', 'EFAV30L', 'ANAK100I', 'ACYC200', 'CAS30L', 'MORP1-5', 'GALA4', 'CIPR200PM', 'FIDA200', 'BACICAP', 'PENT50S', 'VITB1', 'TRI125', 'TRIAM30C', 'DACA200I', 'ALIS150', 'PROB500', 'CYTA2I', 'FENO54', 'NS100G', 'TOBR80I', 'MAG2PM', 'GLYO65OT', 'EPO2I', 'LINE600I', 'DEXA10I', 'ZALE10', 'PILO4ES', 'SNLR215C', 'REPA0.5', 'DEXE5', 'VANCOD125L', 'LIPIDSTUDYG', 'ATOV750S', 'NS2000', 'ETHO50L', 'AMIT100', 'FERR325', 'PREDN5L', 'DECI50I', 'DOLA12.5I', 'DAPT500I', 'GENT120FD', 'DEXA4I', 'MAGMEQV', 'NITR50', 'PRED1ES', '1/4NS100', 'VALGA450', 'ATRA50I', 'ZOLO50', 'GUAI600', 'LOZ25', 'CILO25ES', 'CEFO1F', 'LEVO0.75', 'CEFA500I', 'NACIND', 'METH2', 'IOPA500', 'METO10', 'GCSF300I', 'RIVA3', 'SW3000', 'TACR1IND', 'CLOP.25L', 'LOVE30AMP', 'MEFOX1I', 'BAG1000I', 'GLUT10', 'LEVO500PM', 'MEDR150I', 'INDO50R', 'BROM25', 'NFAT4', 'ATEN100', 'ALBU17H', 'DRON25', 'GABA100', 'MELP2', 'FLUD50I', 'LEUP3.75I', 'ALEN5', 'VANC250PR', 'QUET25', 'HALO2OS', 'NCEFTI', 'DULO20', 'BECONASEAQ', 'NAPH15I', 'BACT20', 'CYCL1O', 'NS0.1ALB250I', 'ALLO300', 'ZOLE4I', 'OLAN0.25L', 'SIRO2L', 'TRIH2', 'NIAC50', 'ERGO8L', 'CLOP01L', 'ZONIIND', 'BEND100I', 'SW50BASE', 'NVAN500I', 'ISON300', 'THEO100LQ', 'INTE75I', 'TIAZ240', 'METC500', 'NAFC1I', 'NEOENOX30I', 'F2B1ED', 'VITA150', 'NEOPENG2I', 'AZTR2FB', 'QUIN324', 'RISPM0.5', 'LEU100I', 'PROP500I', 'LEVO50', 'TRIA50', 'HEPAPREMIX', 'CARB100XR', 'BACTNEO', 'NICO14P', 'SINE10100', 'FIOC', 'ALPR5', 'NEOFAT', 'LACR7O', 'KCL20I', 'DIAZ5', 'MEGE200L', 'ONDA2SYR', 'NIFE20', 'PREG75', 'PALI3', 'AZIT500I', 'DTO.6ML', 'NTG25', 'COSY25I', 'AMPH200', 'NAPR375', 'DOBU250I', 'OMEPNEO', 'NALO2I', 'KETA100/50NS', 'BUPR3I', 'NICO4L', 'METH10I', 'TETA250I', 'KAYE15P', 'CAFCIT', 'SYRIT10', 'SERT50', 'CYCL0', 'MULTNEO', 'MIDO25', 'MEMA5', 'EMYC200L', 'CARB450I', 'ARIP2', 'THIO15I', 'CHLORO30', 'PROC10', 'KENA40I', 'ADAL90', 'ANTI500I', 'D10W250I', 'NICO2L', 'TRUS2S', 'BUPR150XL', 'SALS500T', 'VIABAG', 'VANCPR', 'VAL250L', 'AZAC100I', 'MESADESEN12', 'DOBPREM', 'ISON100', 'METR75G', 'HALO2L', 'THIA200IND', 'LEVO175', 'SULF500EC', 'BUPR150', 'TBDX3.5O', 'PHEN5I', 'NALT50', 'OXYC20', 'TACRSUSP', 'LID2PREMIX', 'FENT2.550', 'PEMO375', 'CEFT1F', 'CISP50I', 'GENTEALO', 'TRIAM.025/45O', 'HYDR2/15C', 'PAREIND2', 'CYTA1I', 'PORF75I', 'D4T15', 'TRIH5', 'LITHOBID', 'TEMA15', '1/2NS250I', 'PARO25CR', 'TAMS.4', 'TETR250', 'VIOK', 'CLAD100I', 'MEPH32', 'ATOM40', 'PEGINTE150I', 'THRO5T', 'DABI75', 'NEOCHL', 'SLOBID', 'ACET125', 'OXAL50I', 'RAMI125', 'ANAMSTUDY', 'CAGLMEQ', 'DIGO.5I', 'PYRI60', 'CORTFOAM', 'ZENA200I', 'PROT250I', 'FELB400', 'OXYT10I', 'TRET50L', 'CLIN900I', 'MESADESEN13', 'FERRG', 'HEPAR10I', 'TRAN10', 'CLON1', 'DOPA400I', 'ATAZ100', 'ALBE200', 'NEBU2500I', 'INSUPUMP', 'FLUC200', 'SW100', 'CHL500I', 'LIDO5O', 'TOLV15', 'PIOG30', 'MICA50I', 'BTL100', 'DILT30', 'NDEPHBAG100', 'GENT20I', 'QUIN300', 'GAMML30I', '*NF* ANAS1', 'WARF10', 'SNLR115C', 'MELP50I', 'QUET25L', 'BACTDES5', 'D17.5W250I', 'CROM4O', 'EFAV20L', 'ARTTS', 'WARF5', 'DEXAM20I', 'VENL75XR', 'TACR1BN', 'RENA500IR', 'AMOX250L', 'CVVH2000I', 'FERRG325', 'GENTFORT', 'CYCL10', 'CREO10', 'D5250.1ALB', 'AZAT50', 'PROP225', 'DANT20I', 'NAMP500I', 'ETOM2I', 'CLIN300L', 'DEXA5L', 'PANC5000', 'MEBE100', 'NEOEPO3I', 'MORP2-10', 'CIDO75I', 'PENT300H', 'NYST100P', 'NAPH50I', 'FLUP10', 'PROP15', 'SNLR260C', 'THIX10', 'MORP2I', 'OXYC5', 'ASCO250I', 'GENT80I', 'NEORAN', 'BECO67H', 'CSA25', 'NMORP', 'NEOMERO', 'PACL300NO', 'FURO100I', 'ETRA100', 'SEPLORA2I', 'PSEU30', 'VANCOIT', 'MORP0.5-4', 'GLYB5', 'BUSP5', 'CEFT600I', 'NEOR25', 'MYCO200/1L', 'LEVO300', 'NELF625', 'FENT75P', 'DORZ2S', 'AMPVL', 'ONDA8ODT', 'MORP250I', 'RIZA10', 'ARIP1L', 'ADACEL', 'FLUV50', 'MORP4I', 'NEONEOIH', 'BAG50', 'CLOP7.5L', 'ASAEC325', 'FLUP125I', 'PENV500', 'NEBU1000I', 'MERC50', 'GABA400', 'PROP20', 'DICY20', 'ISOS5SL', 'PRIM263', 'FELO20I', 'CHARC120L', 'GRAN1I', 'NELF250', 'D5W500G', 'ACYC40L', 'D5W0.1ALB250', 'CEPH250S', 'TRMP200', 'MORP15', 'SIRO1.5L', 'MTX1000I', 'HEPA0.5I', 'MECH10I', 'BENA10', 'INFL0.5LF', 'KBIC25', 'canc70i', 'NTG10P', 'MYCO500SM', 'PANCEC', 'LCARN330', 'VORI200', 'AMLO25', 'BALS750', 'SERT25', 'VANC1F', 'REOPI', 'LITHI150L', 'PERP8', 'COSO5ES', 'AMIN5', 'CROM800H', 'HYDR0.5O', 'NEOBIC', 'GABA300', 'VENO100I', 'CHAR50L', 'MESADESEN6', 'T3', 'MINO50', 'TICA90', 'AMIT50', 'ACE500', 'MEMA10', 'SUCR2R', 'AZTR1000FB', 'OLAN5DTB', 'PSYL1W', 'SYRINGESW', 'LIDPF2I', 'LEVO50I', 'PROPLA80', 'OSEL30', 'VITEDR', 'FLUO2500I', 'NFAT8', 'INDO1I', 'GLIP10', 'DEXA0.5L', 'GLUC10XL', 'COCA10T', 'VALP250L', 'PILO1O', 'LEVO100', 'VERA40', 'CLOT10', 'KAYEX50R', 'DDAV1ND', 'RIVAR15', 'TRAM50', 'MORP100PM', 'NIFXL60', 'AZAT50L', 'ALBU5500NF', 'TIZA4', 'FENT50P', 'NEOPHEN', 'FOSI20', 'ACY500N', 'ADEF10', 'MINO10', 'PROP10', 'PIPE3I', 'HYDC200', 'PYRI1/100L', 'DARV100', 'INDI400', 'PRAM1', 'GLYC1R', 'LEVO.75', 'MORP250D5W', 'NEVI50L', 'AA5D151000I', 'CHLOR250', 'PHEN100I', 'ISOS5', 'LIDOVISC', 'EPO4I', 'THIAM100I', 'PROZ10', 'INRHI', 'ZAN15L', 'PHYT1I', 'BETAVAL0.1/15C', 'EPINEO', 'AUGM0.0125L', 'PANCSOLN', 'OLAN5', 'PARO30', 'NOVO70/30', 'ALLO100', 'KETA10050', 'ERYT250', 'DRAM50', 'H1N1VACC', 'ESTR1', 'PSND', 'CREON12', 'LOPE2L', 'TDT5I', 'DILA100', 'KETA500I', 'THYR60', 'LORA5', 'DIGO100C', 'QUIN260', 'CARB100CR', 'DIPH50I', 'LEVO750PM', 'BACIEO', 'MPRED4', 'NIAC250SR', 'CLAR250SYR', 'MERODESEN', 'METOSUSP', 'KALETRA', 'NALO6L', 'alum454', 'NEFA150', 'THIO10', 'ACET6I', 'SUMA50', 'QUET50XR', 'CEF750I', 'ATOR40', 'VICODIN', 'NICA500', 'NABC50I', 'OMEP20', 'TAXO30I', 'GLYC17', 'ENTA50I', 'KETR30I', 'ACAM333', 'DEME300', 'METF500XR', 'BACT2022G', 'ETHAN70I', 'BETA25ES', 'PRED20', 'MORP4-6', 'RABI10I', 'ADDE5', 'RABVAC', 'MAGN400', 'ACET100L', 'KETO15I', 'PREV.5I', 'MIDA100', 'ATORSTUDY', 'BEVA400I', 'SOLN3', 'ALTE20I', 'TOPR25', 'AMOX250', 'FENT50I', 'BUPI0.1EPD', 'MESADESEN3', 'QUIN20', 'DIGO200C', 'NEOFOL', 'ACGEL80I', 'TRAN2', 'MAG2SYR', 'LOTR5/20', 'LISI20', 'CLON1P', 'ASTR5000I', 'DUPLEX50', 'NEOCHLORO', 'PENV250', 'NICABAGSODCHL', 'ISOS10', 'HYDX10', 'PEGI180I', 'VANC1.25L', 'HEPA10SYR', 'MIVA20I', 'MYCO500I', 'METH10', 'MYSO250', 'DOXY50', 'FENT2.5250', 'DORI500I', 'HYDR25O', 'LANS15L', 'HYDRNEO', 'GEOD20', 'LEUC350I', 'DORN2.5PL', 'DOX2.5', 'METH1I', 'DOCU100', 'PRED5', 'PAPA30O', 'AUGM0.125L', 'D5W250G', 'OCTR500S', 'OXYC10', 'PROC500', 'FENT2I', 'CARB200', 'PREG25', 'OLAN0.5L', 'CAFFCIT60I', 'HYDR10I', 'ENTA200', 'ENOX100I', 'HURRSPRAY', 'NAFC1000I', 'CAMP200I', 'NALO2L', 'KCL20P', 'MESA400', 'DOXA2', 'CIPR400PM', 'THEO400TR', 'HPGB5PB', 'ARIP5', 'MYCO1000L', 'PROC100I', 'PERM1', 'LETR2.5', 'NORE.35', 'SSD400C', 'MYCO750L', 'KYTR1', 'CARV625', 'VANCODESEN', 'PHDOSA90L', 'TRET.025C', 'COZA25', 'CLOM50', 'TACR0.03O', 'DURA100I', 'NAPR250', 'NALO3L', 'EPLE25', 'CALG1I', 'SIRO0.5L', 'PHEN60', 'LACT250R', 'TOBR12I', 'NS250G', 'STAN2', 'CIPROBASE', 'AZTR1I', 'GENT60PM', 'ENAL5', 'GLYCSUP', 'NS50SYRPL', 'NEOFUR', 'CLOB05C', 'CHLO25', 'LABE100I', 'DOME', 'COLC1I', 'SENN187', 'METD5', 'DONE5', 'PENT300I', 'PHEN25N', 'KALE200/50', 'OXYIR15', 'NORE8/250', 'AUGM500L', 'TOBR20I', 'ISRA25', 'NACL3IRR', 'DIGO25L', 'DESI10', 'THIX2', 'PRIM50', 'CEFA2D', 'ALTE100I', 'QUET6.25QT', 'MODA100', 'INDO25L', 'BUME2', 'PB30EL', 'LAMI100', 'FURO100PB', 'CALC100I', 'AUGM100L', 'MEPE50L', 'BETAEL', 'ERLO25', 'SOTA80', 'FENTSOLN', 'NCLINDIV', 'FLUC150', 'FLUT110H', 'PHEN200', 'KETAIND', 'LACO100L', 'TEMS25I', 'CARA1', 'MORP10V', 'EPO40I', 'CHLORO2', 'ALPR1', 'ATEN50', 'MEGE800L', 'QUIN200', 'VASO20I', 'VERA120', 'BIAX500', 'CHLO20I', 'TRIM10L', 'AMRI100I', 'PALINEO50I', 'MPRED2', 'LAMO25BRAND', 'MEPE50', 'MEPH50', 'PROP80', 'NEFA100', 'LAMO5', 'SINE25250', 'PERIC15L', 'OXYB10XL', 'BACI100000L', 'PROC50', 'GENT120PM', 'HPGB5I', 'BENGAY2O', 'ADVA100/50H', 'BLEP10ES', 'BENZ1I', 'AMOX500', 'METH25PCA', 'LITH150', 'NEOVITE26.7L', 'BETA25D', 'VANC.125L', 'TACR0.5BN', 'THEO300', 'AMPIDESEN', 'CHLQ500', 'EPO10INESRD', 'HYDR1SYR', 'MYCO250BN', 'SULF200', 'AMIN35', 'PHEN2.5O', 'SCOP4I', 'MORP2-5', 'HYDR10L', 'ASCO500L', 'GCSF480I', 'METRVAG70G', 'PRED5L', 'CEFU250', 'CIME400', 'CITA2L', 'TRAN4', 'D4T20', 'METD10', 'IMIP25', 'LISI10', 'MAG2PMLF', 'LIDPF1I', 'VIAL300', 'BUPI0.2EPD', 'CHOL4G', 'SALS750', 'AZIT100L', 'PACL300I', 'NEOM500', 'AMIKINH', 'IFBETA30I', 'ACET120R', 'METH10L', 'ACET25T', 'BACTDES2', 'TRIAM.025/15O', 'CEPACOL1', 'PENT400', 'CLIN75L', 'KACE200I', 'LEVO200IV', 'ROFE12.5', 'ALBU3H', 'PCN250S', 'ASA600R', 'TSMP', 'RANI150', 'MORP500I', 'CIPR250', 'DEME150', 'EPIDBAG250', 'NEOFLUC', 'D12W250I', 'METO10I', 'PENT250', 'MAALX30L', 'DEXA2', 'GGAC10L', 'CLOM25', 'HYDRNEO10L', 'TERAZ2', 'NS100IT', 'ASPAR1I', 'ATRO20I', 'DOPA200I', 'CARB300ER', 'ABAC300', 'POLYEO', 'MYFO180', 'BETA12S', 'MEPE50I', 'CYCL250L', 'SUNI50', 'MERO1I', 'FLEET', 'ACETSOLN', 'FLUT110HFA', 'CISA200I']

In [5]:
mine = ['NEOVIDAYLIN', 'PRED20', 'PROP100IG', 'DEXA0.5L', 'DOPA400D', 'IVIGBTL', 'METH2I', 'ACETSOLN', 'APRE125', 'SENOKOTS', 'MORP4I', 'D12.5W250I', 'SITA100', 'AA5D151000I', 'CHLO250', 'NEBU1000I', 'TRIM200R', 'BENGAY2O', 'PROP225', 'CAMP200I', 'RABI10I', 'ATOR80', 'CHLORO30', 'DARV100', 'FULV250IND', 'TEGR200', 'MYCO500I', 'APRA5OS', 'ESTR3', 'SCOP4I', 'DIG5P', 'MESA1000R', 'AMIT25', 'PROC100I', 'ADEN90I', 'SUNI50', 'F4B1ED', 'SILD25', 'METO10I', 'MAXISES', 'PERM5T', 'METH50L', 'TERAZ1', 'BENA10', 'MPRED5I', 'AGGRCAP', 'CLOM50', 'AMIN5', 'PRISMAK5000', 'FENT50I', 'FOLI1', 'CEFU250', 'ASCO250I', 'CALC100SI', 'FLUT110HFA', 'PERA200I', 'TROP5ES', 'MYCO200/1L', 'AUGM875L', 'CIDO1P', 'BISA5', 'ALTE20I', 'TAMS.4', 'ORBE5T', 'VANC.125L', 'AMLO25', 'NORT50', 'PHEN50', 'QUET50XR', 'CHLO500', 'CYPR4', 'EPHE50I', 'BUPI0.05EPD', 'RISP1L', 'NITR100', 'METH5', 'MESADESEN8', 'GCSF300I', 'MEPE50', 'ERGONEO', 'CHAR50L', 'NEOPOLYVI', 'MEPH32', 'METP250', 'DIGO200C', 'METL10', 'TOBR300H', 'ONDA4I', 'MORP1-10', 'ESTR125', 'AMIC5L', 'COAL3S', 'NEOSEO', 'CLON1P', 'NEOAQUA', 'LEUP75I', 'MPRED40I', 'HYD12.5PCA', 'ASPAE10000I', 'ASCO250', 'ZAFI20', 'CYAN1000', 'DONN162', 'SIRO8L', 'LIDO5O', 'NEOSYRD5W', 'FERU510I', 'MOXI400', 'CITA2L', 'FOSC24I', 'MEXI150', 'CAPT125', 'CLINBASE', 'DXIG40I', 'CINA30', 'SEND6SYR', 'KAYX60', 'HYD2530C', 'FLUT110H', 'CEFOT2I', 'DIVA500', 'CLIN150', 'CARI350', 'FURO40L', 'AUGM200L', 'PRIM50', 'IBU5S', 'COSO5ES', 'EPI400SYR', 'ERY1I', 'PREG75', 'MIDO5', 'METH500', 'PAMI30I', 'HYDRIND', 'D15W250I', 'MITX20I', 'LEVO750PM', 'MOEX15', 'MEPI1I', 'THYR120', 'ATAZ100', 'CYTA100I', 'WARF75', 'STLTP', 'CRIT6T', 'CLAR250SYR', 'ROPI0.2/100I', 'ASCOR504L', 'VANCOIT', 'DOXO20I', 'DIVA250', 'CEPH250', 'METH250I', 'RHIIND', 'SYRINGENS', 'FLUCDESEN', 'EPI1N', 'ACEB200', 'PHEN25ES', 'FOSA150IV', 'POTA10PM', 'ROFE12.5', 'METOSUSP10L', 'TOPR100', 'DIAZ10I', 'TACR5BN', 'AMANT50MG', 'HEPA10SYRN', 'CLOP1L', 'HEPA10S', 'LITHI300L', 'ROSU20', 'DEFE500IND', 'ERGO400T', 'HYDR12.5LF', 'CALGLUCNEO', 'LACT250R', 'MESADESEN7', 'NEOFLUC', 'PRAV20', 'GLUL100I', 'NALO2L', 'NESI1.5I', 'FELO20I', 'SIRO6L', 'PRIV5I', 'TIAG4', 'METH20I', 'PRIM1I', 'TEMO100', 'PRED10', 'FOSI10', 'NALO2I', 'DOPA200I', 'METH40I', 'ATORSTUDY', 'ADVA250IH', 'VALP250', 'PROC5', 'ANHC25R', 'MSCO100', 'ETHO50L', 'PANT40IVPB', 'DOCU100', 'GUAI600T', 'TACR5SM', 'AMBI50I', 'HYAL200I', 'PCN250S', 'LOPE2', 'LIOT25', 'OSEL12L', 'ZIDO10L', 'LEVS125', 'THOR10', 'PEME500I', 'THOR100', 'ISOS20', 'CEFT1F', 'LEVO200IV', 'DACA200I', 'DANA200', 'COMBTAB', 'LAMO25BRAND', 'DAUN20SYR', 'PRAZ600', 'OCTR500S', 'THEO80L', 'LIDX15O', 'LEPR50I', 'LAMO5', 'MORP20R', 'BASIS', 'KETA500I', 'PILO6ES', 'PENT300I', 'KCL20I', 'ISOS2.5SL', 'BUPI0.1CADD', 'CEFE1000I', 'MORP1-6', 'SODCIT3SYR', 'TICA90', 'DECI50I', 'LOZ25', 'GOLY60L', 'CEPH500', 'NEOMOR5', 'SUCR2R', 'LEVOX125', 'ETRA100IND', 'THYR1', 'DANM25', 'NIFXL60', 'DOBPREM', 'CALG500', 'OLME20', 'DIDA250', 'TELM20', 'BUPI0.2EPD', 'METZ50', 'GANC250C', 'RISP1', 'ARIP9.75I', 'CYTA1I', 'BENZ5', 'SECR16I', 'ANAMSTUDY', 'ATRO1/15ES', 'CYCL2O', 'MORP1-5', 'MEXI250', 'FIDA200', 'CLOB0.05S', 'PHYT10I', 'NAPR375', 'HYDZ25', 'VERA5I', 'NALO6L', 'INRPI', 'TRIM100', 'SALS750', 'BENZ2', 'OXCA60L', 'LEVO500PM', 'NADO20', 'IOPA500', 'NATE60', 'AMPH500I', 'RISE35', 'BACT20', 'CALC.005C', 'LEUC25', 'MESADESEN4', 'PRIM', 'ANTI500I', 'SEPKET10I', 'ALTE100I', 'GLIP5', 'IBAN3I', 'TEGA2', 'FIOC', 'BENZ1', 'INTR10/250I', 'PROP15', 'MESA400', 'THEO100', 'MYCO2C', 'CLOP5L', 'AZIT100L', 'ZIPR60', 'HYD50PCA', 'MORP250PM', 'SIRO0.5', 'GENT3EO', 'VANCOINV', 'HYDU500', 'DILT90', 'NEOM500', 'FLUP2.5I', 'HYDRNEO', 'FTY7201.25IND', 'H1N1VACC', 'DOXA1', 'IBUP200L', 'FURO100PB', 'AZAT50L', 'AMRI100I', 'HYOS.375', 'CYCL1I', 'EFAV20L', 'NALB10I', 'FINA1', 'GCSF480I', 'ACET100D', 'ILOPDOSEPACK', 'CEFP100', 'LIDEPF1.530I', 'AUGM500L', 'LAMI10L', 'TROS20', 'PERI', 'GLYB5', 'SEVE800', 'MESADESEN5', 'DEXA5L', 'OXAL50I', 'NEOLOPE2L', 'PSYL1W', 'HALO50I', 'CAPT50', 'PILO5ES', 'THYM25I', 'LIDO1BUFF', 'DYAZ1', 'DAKINHS', 'CYCLOP10L', 'SLO300', 'SW100', 'DILT30', 'KET15C', 'MEMA10', 'THIAM100I', 'ATRO1I', 'THYM25IND', 'MISO200', 'HALD50I', 'MEDR25', 'LACO200I', 'RASA0.5', 'PEGINTE150I', 'PYRI60', 'TRUS2S', 'ONDA2SYR', 'LOXA10', 'ANTI250I', 'RITO80L', 'GANC500I', 'EPO2INESRD', 'MORP2IV', 'BUME2', 'TRIF2', 'DIDA200EC', 'MINO50', 'POLYB1MUL', 'THIO15I', 'GENT80PM', 'QUIN260', 'OXA10I', 'CHLO25', 'NICO2L', 'TIAZ240', 'ATAZ150', 'VANC125L', 'ANAK100I', 'INSU500I', 'MAG2PM', 'MICA50I', 'DAPT500I', 'DOXE0.5', 'ACYC40L', 'CISA200I', 'EFAV30L', 'LORA10', 'SUMA50', 'PRAM125', 'DILA100', 'PHEN10ES', 'NEOLEVO5', 'STAV30', 'DEXE5', 'NEONYST5L', 'HYDR2', 'LORA5', 'NIFE10', 'POLY30O', 'NEPH2.71000I', 'BACTSSSM', 'TPN', 'FOND2.5I', 'ASCO500', 'VALC60L', 'NYST100P', 'LOPE1/7.5L', 'NORESTFE', 'FLUV20', 'TBSYR', 'NAAC20I', 'PROAIR8.5H', 'SULF100', 'T3', 'TRIA50', '5000MLBAG', 'CITA20', 'CLOT10T', 'FLUN1H', 'CITR6T', 'NORMOCARB2', 'PHEN25N', 'QUET25', 'OXYM1I', 'CIPR250', 'PROP120', 'MAGN400', 'LEVE500BN', 'URSO300', 'PHDOSA90L', 'SULF500', 'TRIAM30C', 'VANC500L', 'COSY25I', 'PALI6', 'SYNE500I', 'CORTFOAM', 'CETI10', 'NEOSES', 'NEOSYRNS', 'DTO.6ML', 'AZTR1000I', 'ASA81EC', 'NEOCHL', 'THIA200IND', 'ACI25', 'MORP250I', 'HEPA1000/500', 'CIPR200PM', 'LACO200', 'LEVO0.75', 'ESMO2PM', 'INLHI', 'RALO60', 'RISP2', 'DOPA400I', 'MERC50', 'CEFU75I', 'NCEFTI', 'ACYC200L', 'ROPI2', 'ISOS5', 'TRIM10L', 'DIAZ5', 'AMOX250', 'ZOSYNDESEN', 'ADDE20XR', 'TACR0.5BN', 'DESI30O', 'PANC5000', 'GALA4', 'DEXA5', 'OXALDESEN', 'SULI150', 'FLUT1C', 'CREO10', 'SERT50', 'AMIN500', 'TILAIND', 'DROTIND', 'DDAV1ND', 'LAMO100BRAND', 'SW30SYR', 'TRMP200', 'MALA250/100', 'IPRAPF', 'COLE625', 'DABI150', 'FOSI40', 'NIAC500', 'GLYB1.25', 'KETO2SH', 'PENT400', 'MISO25', 'CALC.5', 'VIABAG', 'PRAZ2', 'MCT30', 'NAFC1000I', 'RIBA200', 'MTX200I', 'CLOZ100', 'ERYT250', 'KETO15I', 'CLOP.25L', 'KETA100SYR', 'BAG1000I', 'OXAL100I', 'LIQUIDE', 'PROP40', 'LEVOBASE2', 'NPENI', 'MONT5', 'HALO5', 'DEXA4', 'MEGE800L', 'PROP500I', 'CATA1', 'ETID200', 'TERI750I', 'NEOSI', 'AMAN100L', 'AMPI500', 'IBUP600', 'ACET6I', 'ISOS40SA', 'MORP25I', 'LEVO1.5', 'PHEN2.5O', 'FMLSUSP', 'SILV1C', 'BACTDES5', 'VASOAES', 'ACE250', 'LANS15', 'ROMI250I', 'RANOL500', 'HEPA0.5I', 'TALC', 'COLC6', 'PYRI50/25L', 'LITHI60L', 'TRIF12.5I', 'AMIKINH', 'CHLO10', 'NAMP50N', 'DEXA15', 'ATRO20I', 'ARIP15', 'MELP2', 'KETA5I', 'NIMO30', 'NEOMERO', 'MESN1I', 'INTE10I', 'ERLO150', 'VOLU450L', 'SERT100', 'QUET200', 'QUIG324', 'INSULINCARB', 'TERB1C', 'NIFE20', 'VENO100I', 'CEFA5ML', 'BETA0.05C', 'LR5SYR', 'BALS750', 'PRED0', 'PIRO10', 'GLU5/500', 'THEO400TR', 'ETHAN70I', 'CLOB.05C30', 'H2B1ED', 'TOPI25', 'AMIT100', 'LEVO150', 'LEVO75', 'MAAL30BTL', 'DEXA10I', 'MENI1SYR', 'KALE200/50', 'OLME10', 'PILO2ES', 'SINE10100', 'BICA50', 'NEOR25IND', 'EXEM25', 'PRAV10', 'ZOFR4', 'SELE5', 'RIFA600I', 'NS1000OXYT', 'RIVA15', 'ASA600R', 'DIGO25L', 'METR50L', 'DOLAS100', 'DARB200I', 'MEPH100', 'QUET300XR', 'GLIP10', 'PEDDIG0.1', 'TIOT', 'RISPM1', 'INDO25', 'EPO10I', 'ANUSHCOINT', 'METH125I', 'ANAS1', 'DEXT500B', 'MORP2000I', 'D4T20', 'VITB1', 'FLUR3ES', 'NS100CG', 'MORP1-8', 'INSUPUMP', 'PHEN60', 'RITU500I', 'TRIA10I', 'METH12I', 'ALLO100', 'PERG25', 'MORP10V', 'COMB14.7H', 'DEME150', 'ACY5O', 'ETHO250', 'LORA2PB', 'VITE400', 'EPO20I', 'FOND5I', 'DIGO100C', 'LANS30L', 'FENT10I', 'FLUP25I', 'FLEC100', 'LIDX60O', 'LEVO100', 'ALBU%500', 'DOXO50I', 'CIPR0.3EO', 'EPI1I', 'THIAIND2', 'DIPH2.5L', 'TRIF1O', 'HYDR20/100NS', 'TRUV200/300', 'PYRI10/100L', 'FELOD5', 'CHLO180S', 'GLIM1', 'FLUT16N', 'GGAC10L', 'KAYX30', 'STAV40', 'LITH150', 'NS20SYR', 'HYDR500L', 'PANC10I', 'NALO1I', 'NADO80', 'METR1000L', 'ABAC300', 'COLC5', 'HC500I', 'ENAL25I', 'PIOG45', 'PHEN100', 'HUMU100I', 'BUPR100SR', 'DEXA4I', 'ANAG500', 'CARB100XR', 'LEUC350I', 'NEBU2500I', 'MESADESEN11', 'ASTR5000I', 'FERR300L', 'SYRINGE60NS', 'PROC10', 'MARA150', 'TIZA4', 'SIRO1', 'IMDU30', 'FLUO500I', 'LIND1S', 'MAALLIDO120L', 'ISMO20', 'TOBR80I', 'D54.25T1LI', 'SAQUIN200', 'MAGO140', 'METH20SR', 'CALG1I', 'TENEIND', 'CART1ES', 'BACI25/1L', 'DILHP50I', 'HOMA2ES', 'GLU125/250', 'RILU50', 'PERP8', 'THRO1T', 'NEOCHLORO', 'DILA30', 'ACET325', 'AMIN35', 'QUET100', 'PROP20', 'FLUO2500IVP', 'LINE600I', 'D25W250I', 'NTG100PB', 'TOPR50', 'NYSTC', 'NEOSYRD7.5W', 'PHEN10I', 'DICL75', 'RENA500IR', 'CYCLOBASE', 'LAMI100', 'LOPE2L', 'OKT35I', 'QUET200XR', 'MISO100', 'NFAT8', 'POSA200L', 'FENO54', 'SODCITABL', 'THIX10', 'DAKINHS480L', 'MAALX30L', 'MORP4-10', 'SNLR160T', 'METH2', 'LIDO100S', 'DOXE50', 'TOPR25', 'FENTSOLN50', 'HYDX50', 'NEOCARB200L', 'GUAN1', 'UREA10L', 'PB30EL', 'NERYIV', 'NOVO100I', 'BETA25D', 'SALM13IH', 'DIAZ5I', 'LEV100', 'CLOP2.5L', 'PHYT1I', 'AZIT500I', 'IDAR10I', 'NTG100PM', 'AMBI5', 'NTG2O', 'EPO20INESRD', 'ALPRNEO', 'LORA4L', 'ADAL30', 'CLOP15L', 'TOLA250', 'NEOEOP4I', 'SIME40L', 'DARU600', 'MYCOC', 'SIRO0.5L', 'PEPTO30', 'DULO20', 'CHLO4', 'CHAR120L', 'EPI30H', 'MECH10I', 'CEPHDESEN1/10L', 'TACR0.03O', 'PNEU5I', 'HYDZ20I', 'PERP5I', 'HBIG5I', 'NEOEPO3I', '5FU500I', 'VITA150', 'EPOHD', 'TACR5', 'GLUC10XL', 'SINC50200', 'FEXO60', 'TORS10', 'INSULIN', 'SLOBID', 'EMTR200', 'PENT100C', 'SIRO1SM', 'CANGIND', 'PARO30', 'WARF5I', 'DOX2.5', 'NICA2.5I', 'OCTA10I', 'ISOS5SL', 'DEXA1ES', 'IMIP500I', 'DTOBULK', 'ZOLE4I', 'FLUP125I', 'GLYB25', 'NITR25', 'OSEL30', 'AUGM500', 'DIDA125', 'MAXI1ES', 'PIND5', 'TORS100', 'MILKTH350I', 'NITR50I', 'NEOR100L', 'CEF750I', 'LEVO250PM', 'DAUNO20I', 'LAMO25', 'FENT2I', 'FURO40', 'KAOP30', 'MORP100PM', 'ZAN15L', 'OXYT10I', 'CARB450I', 'PYRI1/100L', 'CEFA2D', 'THYR30', 'BENZ10O', 'BLEP10ES', 'SUMA6V', 'MAG4PM', 'NALT50', 'SODI4I', 'HEGB1I', 'CAPE500', 'CEFP200', 'LEVO125', 'BUPR150XL', 'THEO100LQ', 'DOPA400PM', 'LORA20I', 'HALO2L', 'PHEN12SYR', 'LIPIDSTUDYP', 'CADDNS250B', 'LORA1L', 'DOFE500', 'HYDR10', 'REPA2', 'VITA25', 'MEPE500PCA', 'DIBU', 'FENT75P', 'BAYHEP.5I', 'SILV1T', 'ISON100', 'THIO100', 'ZIPR20I', 'INDCLOF20I', 'FLUP1', 'EMLA2.5', 'LANT750', 'FOSA700', 'PHOS250', 'INFL0.5LF', 'CYAN500', 'FELO2.5', 'CALCIT950', 'TRIAM.025/45C', 'NICO14P', 'HIBTITER', 'BUPR150', 'CROM40N', 'PSEU30', 'SIRO2L', '1/2NS250I', 'NTG4SL', 'BACTDES3', 'PAMI90I', 'NEOINDOM', 'EUCE120C', 'ROSI8', 'PROP50', 'NTG65', 'LINE600', 'CEFX2F', 'LACT500R', 'LEUC5', 'TRIM100I', 'AMOX1000L', 'DIPH50I', 'TERLIND', 'CILOS100', 'PEXEIND', 'PRAM..5', 'LEVO300', 'D5W500G', 'MACR100', 'AMID200', 'GLAT20I', 'BLEP10EO', 'TRIM250', 'VANC500I', 'QUIN10', 'NEOEPO', 'ARTI3.5O', 'CHL500I', 'PYRA500', 'ENFU90I', 'MORP0.5-4', 'EPIBAG', 'BETAH1', 'BETA30I', 'MESNA1I', 'MORP1-4', 'VERA180SR', 'RIFA150', 'FAMO20PM', 'PEPC20', 'PROZ10', 'CADDNS250', 'VITBC1', 'TOBR3ES', 'LANS30', 'NITP50I', 'PAPA150', 'NEOFLUCIV', 'LANS15L', 'MSCO30', 'ITRA10L', 'UREA40C', 'DEPO200I', 'OSEL6L', 'LOVE40I', 'INRHI', 'LABE200', 'AMOX50', 'MAFE5', 'NEOAUGM50S', 'MODA200', 'VANC1.25L', 'ANID100I', 'FLUP5', 'LUBI24', 'TRAN1', 'MORP5I', 'INDO75XR', 'NAMP500I', 'LORA2L', 'DERMON', 'THYR60', 'ACY500N', 'FLUCO400PM', 'ZONI100', 'DIGO.5I', 'DOXA2', 'TREP1.74', 'EPO10INESRD', 'RISP25I', 'ACD3/1000I', 'RIVA10', 'PROM125', 'MAG2SYR', 'MAGMEQV', 'LIDX60C', 'MICO745C', 'PIPE3I', 'RIFANEO60L', 'CARV125', 'DRON25', 'MOLI5', 'GUAI600', 'NEOKCL', 'FEBU40', 'PHEN200', 'ADACEL', 'ACET40', 'GUAIC120L', 'LITH450SA', 'REOPI', 'DELA100', 'AMOX500', 'BIVA250I', 'OMEP40', 'NEOERGO20L', 'MORP20/30L', 'DOFE250', 'PROP20L', 'TRIA25', 'PRAM1', 'TERC80V', 'DIPI1ES', 'MONT10', 'RALT400IND', 'OXAN25', 'OLAN5', 'MELP50I', 'CYTA50I', 'MOVIKIT', 'VIOK', 'HYDR0.5O', 'TEMA15', 'ACET1000I', 'TETRA5ES', 'CLON3P', 'RABI2I', 'TELA250I', 'GENT10I', 'COMBPTCH', 'MEBE100', 'KACE200I', 'CLOB005O', 'ADAL40I', 'CALC13.6I', 'VAP500I', 'CLOB.05O30', 'KCL20PM', 'NICA30', 'NEOACET2', 'KPHO150I', 'IRBE300', 'NORT10', 'BACI15O', 'GLUT10', 'NEORAN', 'ATRO4I', 'D7.5W250I', 'FENT50P', 'CEFDES', 'FLUC200SYR', 'BIAX500', 'PERC', 'LOVE30AMP', 'LAMI250', 'FOSA150I', 'TACR1IND', 'QUET6.25QT', 'D12W250I', 'ATOR20', 'LEFL10', 'NIFXL30', 'MONI7C', 'RAMI125', 'LEVO50', 'POVI30O', 'DOME', 'METO50', 'THIX1', 'GLYC1I', 'NORE1600SYR', 'NOXAI', 'TDT5I', 'LACO50', 'MYCO250IND', 'alum454', 'DIAZ10', 'MOR10L', 'VORI200I', 'SYRIT10', 'MAGSUL20I', 'CLOT1VC', 'EPO4INESRD', 'BECONASEAQ', 'THYR180', 'METO10PB', 'DACL25I', 'MORP20/120L', 'SINC5I', 'NAPR250', 'ANUS1O', 'RANI150I', 'AMP50I', 'CYCL1/2ES', 'MAGS1I', 'OXAC300', 'OXYB10XL', 'CLIN300PM', 'INVE3', 'BTL100', 'SYRINGESW', 'AZAT37.5HT', 'PYRI25', 'HC5', 'LEVO175', 'DIPH125L', 'GLUCTAB', 'NABC50IV', 'NITR50', 'DESMOIND', 'DESI50', 'CYCL50', 'CROM800H', 'MESA4R', 'DICL50', 'VALG50L', 'LORA1', 'PANT40', 'VANC12.5I', 'ENBR25I', 'LEVO50I', 'AMOX125L', 'BETA12S', 'PROP1I', 'ALEN5', 'EMYC200L', 'TRIH5', 'NEOR100', 'NAPH15I', 'ZDVLIQ', 'HEPPREMIX', 'MESADESEN10', 'PSEUD30L', 'LATA5OS', 'GENT120PM', 'MESADESEN13', 'DDI25', 'CHARC120L', 'ILOP6', 'BETAEL', 'ISOS40ER', 'FEN1250PCA', 'LEVO25', 'CEFANEO', 'INDI200', 'LEFL20', 'STRE1I', 'ARIP1L', 'CABE0.5', 'TRAN10', 'ERGO8L', 'MILR20PM', 'GLYCSUP', 'POTA20', 'TBDX2.5O', 'HYDX25', 'DILT125I', 'PROC375', 'LEVOBASE', 'DESO15C', 'TOPI100', 'NEOLEVO', 'DILT500I', 'CANC50I', 'NEOMID2.5', 'CODE30I', 'VALGA450', 'GENT20I', 'HALO2OS', 'OXAZ15', 'DEFE500', 'NEOCEPH', 'ALLO300', 'CHL500PB', 'FAMC500', 'THIO10', 'OLAN10', 'PROC250', 'FLUC200', 'FLUV100', 'QUET12.5HT', 'VITA5I', 'ACE500I', 'GANCIND', 'CEFR250I', 'PARI5I', 'ERYT25L', 'MEPE25I', 'TRET.025C', 'NALO4I', 'ITRA100', 'MIDAZBASE', 'OXACDESEN', 'TIPR250', 'ANUSOL', 'MIDA5I', 'GLIM2', 'HCTZ50', 'BACL10', 'DAPS100', 'CITR30L', 'IFBETA30I', 'LID2J', 'VENL75XR', 'MESADESEN9', 'LOPER2L', 'BUME05', 'NARA2.5', 'ALEN70', 'CARA1', 'TRIAM.025/15C', 'SW50BASE', 'DIDA400', 'D10W250I', 'ROFE25', 'MORP2-6', 'SARNL', 'MORP250D5W', 'ALEM30I', 'LOXA25', 'VITE100', 'QUET25L', 'BUME1', 'SINE25100', 'NEOZDV', 'AMPI250L', 'CISA20I', 'CEPHDESEN1L', 'HYDR25I', 'HEPA30I', 'VITE1000', 'BTL250', 'MEPH50', 'TIGE50I', 'ATEN25', 'CLOP7.5L', 'DEXA1L', 'FISH1000', 'NIAC500SR', 'DULO30', 'ERYT250L', 'SIRO3L', 'PREDN5L', 'OCTR50I', 'ALLO500I', 'AMPH200', 'ASA325', 'TERB1I', 'SNLR215C', 'BUDE3', 'BETAVAL.1/45C', 'NEOPRED2', 'LIDO/EPI30', 'BACTDES2', 'CYPR2L', 'CAPS25C', 'LAMO100', 'WARF1', 'NEOSYRD5W50', 'DDAV400I', 'RISP05', 'HYDR2PF', 'VECU10I', 'FAMC250', 'ALEM30SYR', 'LINE20L', 'PHEN5N', 'DOXY100C', 'INUHI', 'VITEDR', 'METR500', 'HC100I', 'NVAN500I', 'AMIT50', 'ALIS150', 'AMIT10', 'BECL16.8H', 'MODA100', 'METL5', 'BUSU60I', 'FLEC50', 'DORN2.5PL', 'NEOKCL1.3L', 'D17.5W250I', 'METH25PCA', 'DT.5I', 'NELF625', 'PEPC20I', 'TEMS25I', 'ASA300R', 'MESADESEN3', 'FURO80', 'COLO06S', 'CARB200', 'PRAV10L', 'PILO1ES', 'THRO5T', 'MILR10I', 'EDRO10I', 'FENT25P', '*NF* ANAS1', 'ZIPR40', 'ERGT1', 'ALPR500I', 'FLUO2500I', 'ISMO10', 'ALPH0.15O', 'HEPA5I', 'DEME300', 'PREG25', 'BETA25ES', 'SYR1ML', 'ERGO8000LNEO', 'BACTDES4', 'MIRT15SOL', 'CLIN1S', 'RITU100IND', 'FENT20I', 'HEMOSUPP', 'SALM50D', 'CARV31.25L', 'BUPR75', 'OXYC10', 'TERAZ5', 'THIA50', 'BACIEO', 'MESA5S', 'VALS80', 'AZTR1I', 'CHLO2', 'FOLI1I', 'BLEP2ES', 'TOLVIND', 'TERB1SP', 'URSO250', 'DORI500I', 'MESADESEN12', 'AUGM50L', 'METC500', 'ZEASP', 'NHYDRA', 'PRIM263', 'FOND7.5', 'BETA10T', 'TTOX5I', 'ATOR10', 'ETHA50I', 'TDAP0.5I', 'NAPH50I', 'TOBR20I', 'METO25', 'FOSA10', 'FURO40I', 'VASO20I', 'TIMO25ES', 'LEV250', 'NEOENOX30I', 'DISO100CR', 'METL25', 'TAXO160I', 'COLI150I', 'LEVE250T', 'VITA50000I', 'PROP05ES', 'HYDR500I', 'AUGM1.25L', 'OCTR500I', 'ONDA4ODT', 'BACT2C', 'NALO4ILF', 'BETA10O', 'CLON5I', 'ROSI4', 'OXYM15', 'DOXY50', 'INTE75I', 'TOLV15', 'ATOM40', 'FINA5', 'CEFT1000DES', 'NEOPHOS2', 'RISP3', 'THOR25', 'NICABAGNS', 'RIFA200', 'CLON1', 'BROM25', 'AMIO40L', 'FLUC200L', 'CLOM25', 'CLON2P', 'ONDA8ODT', 'METH40', 'PCNGB600I', 'BACI5I', 'AUGMSUSP', 'PHYT5', 'ASPAR1I', 'PREV.5I', 'CORTOTIC', 'VANCO10I', 'SEVE400', 'LORA5L', 'TORS20I', 'COCA10T', 'TAMO10', 'EPIDBAG250', 'CLIN900PM', 'NEOCAP', 'HYDR1SYR', 'DOXE25', 'SIRO1L', 'CEFT2I', 'RIVA3', 'ESTR1', 'GOLY4L', 'VANCOD125L', 'BENZ2.5G', 'ZINC220', 'TEST25P', 'PERM1', 'CALC1I', 'TRAM50', 'THOR50', 'MIVA20I', 'EPIR50I', 'IVIGBAG', 'LIDO1I', 'GLYN3', 'MIDA100', 'LIDO5T', 'GABA250L', 'IMAT200L', 'BETA0.05O', 'GEMT5I', 'TRIF5', 'HEPAPREMIX', 'ALBE200', 'TRAZ100', 'TOLV30', 'MERO1I', 'HYDR0.5C', 'ROFE3I', 'INSNEO1', 'ATROP1I', 'ENAL5', 'IMAT400', 'KETA200I', 'VANCPR', 'LIPO20200', 'OMEP20', 'CEFU750F', 'D20W500I', 'KETO200', 'NAAC100I', 'MTX25', 'canc50i', 'PROC750SR', 'VALS40', 'PROG100', 'VALG450SM', 'DOFE125', 'MEFOX1I', 'RISP37.5', 'TENO300L', 'NCLINDIV', 'FLUC100', 'FTY7202.5IND', 'BECO67H', 'PEGA75I', 'REFRESH', 'METH1', 'SENN187', 'RITO100', 'HEPA5000I', 'TEMO5', 'BACT2/1O', 'DIGO125L', 'CEFT2F', 'FENT250I', 'REPA1', 'ESTR25I', 'BUPR2', 'AZAT75', 'OLAN0.25L', 'IMAT100', 'OSEL75', 'NTG25P', 'OXYIR15', 'NEOCAL', 'OXAC1I', 'PYRI60SYR', 'APAP80R', 'TOBR3EO', 'ALTE1/2I', 'OXAZ10', 'NAFC1I', 'ERGT1R', 'TIRO12.5/250', 'PHEN10', 'PHEN250I', 'TACRSUSP', 'PENDESEN', 'COMP10I', 'PHEN130I', 'CALC667', 'PRED5', 'SW5SYR', 'TIZA2', 'PANCSOLN', 'NEOSYRD12.5W', 'MEGE40', 'DOXA4', 'NEOSYRSW', 'ENTE0.05L', 'METH8', 'HCL1MEQ', 'PERP4', 'NAVA250I', 'CIPR750', 'HOMA5ES', 'METF500XR', 'ALTE1/5I', 'MAG20/500', 'UNASNEO', 'NEOMETO', 'TACR0.1O', 'FOSP500I', 'WARF2', 'MOEX7.5', 'AMOX250L', 'FULV250I', 'LOVA20', 'CHLO500L', 'MIDA2I', 'POLY15O', 'TRIA.1/80C', 'ROPI5', 'CAND16', 'TEGA6', 'CAFCIT', 'ALTE1I', 'GENT20IVT', 'METC750', 'NS0.1ALB250I', 'FENT2.5I', 'CARB200L', 'HYDZ50', 'ASPIDESEN', 'NOVOR100I', 'DOXO50SYR', 'INDI400', 'PAPA30O', 'SW3000', 'RIVAR15', 'INDO1I', 'HEPA10MUI', 'RITA10', 'CALC400S', 'NTG15P', 'ACET0.04IR', 'SUNOLIND', 'LIDO1/10I', 'CATA2', 'SOLI5', 'NOVO70/30', 'MULTNEO', 'FERRG', 'AQUA400O', 'TIC250', 'TIAZ120', 'NITA500', 'TACRSUSPBN', 'FENT2.550', 'EPO4I', 'KYTR1', 'RUFI200', 'NPHEN10I', 'ARIP5', 'FENTSOLN', 'NTG5P', 'CLIN900I', 'TENO300', 'LOPE1L', 'VENL375XR', 'SODI650', 'ASAEC325', 'CYCL250L', 'NYST60LSM', 'AZIT600', 'PHYT4L', 'HYAL150I', 'PENT50S', 'BAG50', 'VALS160', 'NS1000IBTL', 'EZET10', 'LIPID250I', 'CIPR400PM', 'MEPE50I', 'OCTR1000I', 'NIAC50', 'THIX5', 'NEUT', 'ZOLO50', 'KBL120L', 'TIAZ300', 'MORP1000I', 'ENTE1', 'TEST5P', 'CILO25ES', 'INSNEO', 'LACT12T', 'ACCUSPAY', 'NEOZOSYN', 'RIFA300', 'SIRO7L', 'ATAZ200', 'NAPH1O', 'OLAN2.5', 'NS50SYRPL', 'KALETRA', 'LIDPF2I', 'CLOZ25', 'MECL125', 'MORP2-10', 'HC20', 'CSA50I', 'THIA100', 'MORP500I', 'RABVAC', 'NEPH1', 'INTEG20I', 'SOLN3', 'LIDO2/5J', 'LURA40', 'DIPY25', 'AMOX500L', 'GENT80I', 'AMIK500', 'ADDE5', 'DAPI0.5ES', 'PB60I', 'CEFT600I', 'FORT200', 'AMPVL', 'ADEF10', 'TBDX3.5O', 'KAYE15L', 'CEPH500L', 'COSO10O', 'NEOACET', 'PROZ20L', 'SILD20', 'SWSYR20', 'TRDI40I', 'METR1G', 'FENT2.5250', 'FLUM1I', 'ROPI1', 'MIDA50I', 'CVVH2000I', 'CIPR50L', 'CLIN600I', 'DIDA100', 'DROT5I', 'MORP15', 'ACE500', 'PHYT2L', 'HYDR1I', 'OXYM0.25', 'NATTTS', 'KENA40I', 'TERB25', 'MAG2PMLF', 'TRET10C', 'PIMO2', 'LANS30ST', 'QUIN325', 'SCOP.25ES', 'SIMV10', 'ROPI0.2CADDB', 'DIPY50', 'VANC12.5L', 'MIDO25', 'NALO3L', 'ALBU17H', 'LEVO200', 'TRET50L', 'PROP10', 'OXYC20', 'MINO100I', 'MORP30', 'ASCO1000I', 'CLIN75L', 'OLAN5DTB', 'ATEN100', 'CACHL30L', 'MORP4-6', 'FLUP10', 'LEV500', 'ENTA200', 'CALC100I', 'GLYC17', 'DEXTROSE', 'VANC.000125L', 'PHEN30', 'KCL40I', 'PROP200I', 'ACYC1000I', 'NAPH15O', 'PROPLA80', 'MORPH100', 'ALBU25IVPB', 'QUIN800I', 'ROSI2', 'UNAS15I', 'HYDR2/15C', 'MIDA5050', 'FLU200PM', 'SORA200', 'AMAN100', 'PHYT3L', 'ETOD200', 'AUGM0.0125L', 'LIDO5O3.5', 'PARO10L', 'CEFO2F', 'GENTEALO', 'TOL1C', 'PARO12.5CR', 'MICROK10', 'TOLT2', 'MSCO60', 'D5250.1ALB', 'DARU600IND', 'NORE.35', 'BACI100000L', 'NEOTRIVIT', 'FLUC150', 'D5W250G', 'METR250', 'EPI30I', 'ESOM20', 'OLAN10ODT', 'IBUP400', 'MORPITPACU', 'NEWMED', 'FURO10L', 'NORT10L', 'GABA100', 'CIME400', 'EFAV200', 'D550.2ALB', 'ACYC200', 'KAYEX30R', 'NEOM30O', 'INTEGR', 'LEVE500I', 'PROC500SR', 'NS100H', 'FLUD50I', 'EPOP5I', 'AMLO5', 'PANT40SM', 'BAG250I', 'CAFF250I', 'HYDC200', 'SIME80', 'ACET125', 'PIOG30', 'POSA40L', 'FERR62.5I', 'ONDA4', 'DIAZ50L', 'MYCO1500L', 'MILR5I', 'CYCL25', 'HYDR2I', 'BUPNAL82', 'QUET50', 'CHLQ500', 'CISP50IV', 'SIMV40', 'NABU500', 'NOREBASE', 'SOLN2', 'PREG5L', 'VORI200', 'MORP2I', 'PRIM250', 'TAXO30I', 'CIPRO400I', 'PROT50I', 'TUBE5I', 'ANAL15C', 'EPLE50', 'BETAAUG0.05G', 'AMPI500L', 'COZA25', 'HYDM4', 'METHB1I', 'ETOP100I', 'PRED5SM', 'PRAM.25', 'ALBU5500NF', 'LITHOBID', 'EPINEO', 'PYRI60L', 'TIAZ180', 'NEOPROP4', 'TRI125', 'TRETL', 'PERCBULKL', 'NEONEOIH', 'RIVA2L', 'TRIH2', 'LEVE100L', 'SNLR260O', 'DIVA125SP', 'DIPH25', 'RISPM0.5', 'ENAL10', 'PYRI180', 'D4T15', 'DIPY75', 'BISO5', 'DIPH25L', 'CLOF20I', 'METH10SY', 'ASCO500L', 'NIFXL90', 'NICABAGSODCHL', 'ABAT250I', 'STARTPND10', 'INNHI', 'FULV125I', 'PANCNEO', 'PIOG15', 'NOVONPH100I', 'MEGE40L', 'HCTZ25', 'SULF500EC', 'CYTA2I', 'INTE15I', 'IFEX1I', 'PENT300H', 'MERO500I', 'LITHI150L', 'OLAN7.5', 'CLOP05L', 'NEOSYRD10W', 'PRED125ES', 'MORP2-4', 'MINO100', 'PRED5L', 'SNLR115C', 'TRAN2', 'CHLORO2', 'MARA300', 'NS500IND', 'PALI100I', 'FLUD100', 'ACET650L', 'GANC500', 'CYTA2000SYR', 'HALO1', 'PARO20', 'AMIO150I', 'SIRO1.5L', 'SOTA80', 'LOTR5/20', 'DESI2C', 'DARV65', 'KCL20P', 'REPA0.5', 'HYDR10L', 'INHRIV', 'GENT3ES', 'DARA25', 'PROC5L', 'CARB25', 'AZTR2I', 'PROB500', 'LORA2I', 'CANC70I', 'CAPT1', 'CLORA7.5', 'TRAN4', 'ATOR40', 'TRET.05C', 'PROC500', 'ATEN50', 'IMIP25', 'ESCI10', 'ATOM60', 'METF850', 'PYRI10I', 'PILO5', 'MICO2P', 'EXCELNS1000', 'TRIA1P', 'HALO2', 'MEMA5', 'BLISTEX', 'MEXI200', 'HYOS.125', 'AUGM875', 'CYAN250', 'ARIP10', 'AMID100HT', 'NEOBIC', 'AZAT50', 'AMPH250I', 'ACDA2.5I', 'NABU750', 'STARTNEOD5', 'OMEPNEO', 'DISO100', 'THEOP125', 'HEPAR10I', 'ESMOB100', 'SIMVIND', 'SULFA500', 'PHENO100', 'NFURON', 'DICL250', 'INFL100I', 'NYSTO', 'NIFR150', 'CODE30', 'ACAR50', 'LETR2.5', 'HALO10', 'MSCO15', 'ATOV750S', 'RANI150L', 'FENT12.5P', 'MESADESEN1', 'CYCL1IV', 'LEVO.75', 'CIDO0.25S', 'LEVO750', 'ATGDESEN', 'RAMI5', 'VERA40', 'CEPHDESEN50L', 'SEVE403', 'NS2000', 'MIRT15', 'KBIC25', 'RIBO50', 'ALTE2I', 'GLU25/500', 'BACT2022G', 'AMIC250L', 'KBL240L', 'FENT100P', 'FLUC60L', 'PHEN65I', 'TSMP', 'DOXY100I', 'LEVO4I', 'VINB10I', 'CAL1250', 'IODO210', 'TOLC100', 'TEST4P', 'OXYT20/1000', 'ZOSY2000I', 'NEOEPO10I0', 'HYDR1AMP', 'ENTA50I', 'DRON400', 'ENAL25IVPB', 'HYDRO100I', 'BETH5', 'GLAR100I', 'RITO100T', 'DESMIND2', 'MEGE200L', 'UROC10', 'MEPE50L', 'CIPROBASE', 'NEOR25', 'PROC10I', 'ISOP1I', 'AZAT25HT', 'HYDRNEO10L', 'NACL4I', 'DAKINES', 'PARO10', 'BACTSS', 'BACT2/15C', 'DEPA500ER', 'LANT250', 'DROP5PB', 'VITAMA10000', 'ASAB325', 'FLUV50', 'DEFE500I', 'VANCODESEN', 'HER440I', 'PHEN50I', 'BACTNEO', 'PROM25', 'VANCO25O', 'NTPN', 'SULI200', 'BENZ1I', 'COENIND', 'METH1L', 'MANN25IV', 'NEOURS', 'IRBE150', 'GRAN1I', 'ABAC20S', 'POLIOV', 'CETX2I', 'NEFA100', 'THIA100I', 'FENT12P', 'LACO50L', 'DILT60SR', 'METOSUSP', 'LEVO88', 'IPRA2H', 'PRISMASOLK32', 'NICA500', 'SW1000', 'CLOB.05G30', 'DESI100', 'LEVO100I', 'GLYC1', 'CSA100', 'ISON300', 'PRED50', 'MARIIND', 'ACYCIND', 'MERODESEN', 'NFAT6', 'CAND4', 'FERR325', 'DIGO125', 'INSU100SYR', 'CYCL100I', 'HYDR10I', 'ACY500I', 'CEFT500I', 'KAOL60L,', 'NICO7P', 'THAL50', 'PROP500IG', 'MPRED1I', 'PGBNEO6I', 'DARU400', 'NTG25', 'ACET1000PB', 'HYDR1C', 'VERA80', 'ATRA50I', 'HEPA10000/5I', 'D5W0.1ALB250', 'AUGM250L', 'BETA0.05L', 'NORT25', 'TREP20I', 'LISI5', 'ZALE10', 'THIO500K', 'AZTR1000FB', 'OXYB5', 'MYCO250BN', 'PERIC15L', 'MYCO500T', 'ENOX150I', 'TRAZ50', 'PACL300I', 'PRAZ1', 'DOLA100I', 'LIPIDSTUDYG', 'FENO10I', 'MYCO500SM', 'LEU100I', 'ACET100L', 'DEFE2000I', 'PROP200IG', 'ETHA100', 'VARI125I', 'CYCL1O', 'PRAZ5', 'NIAC250SR', 'TRIAM.025/15O', 'nmetroni', 'NEOFUR', 'NEOVITE26.7L', 'HYDRNEO1L', 'HYDR1000L', 'NTG3SL', 'H1H1VAC5I', 'FENT5I', 'BETAVAL.1/15O', 'AUGM250', 'ZON100', 'EPLE25', 'VENL75', 'DAPS25', 'MYFO180', 'NE1/35T', 'CLOP01L', 'HYDX10', 'METF500', 'MYSO250', 'CEVI30', 'NABC50I', 'EPI15H', 'QUIN40', 'DICY20', 'HC100IC', 'MAXZ1', 'PENV250', 'NITR20SYR', 'DISO150', 'FLUC200SM', 'NS100IT', 'BENZ100', 'ENTE0.5', 'AFRIN3ML', 'BACTDES1', 'LABE100I', 'FEN625PCA', 'BAG100', 'COLC1I', 'VICODIN', 'SEPLORA2I', 'NACL1', 'ASPI325INDP', 'METRVAG70G', 'GENT60PM', 'HALOPB', 'DOCU100L', 'CLO025L', 'AMIL5', 'QUIN20', 'INDO50', 'NEOPREDL1', 'TERAZ2', 'CSA100L', 'MECL25', 'ZONIIND', 'VALDIND', 'RITU100I', 'NAVEL10', 'DAKINSF', 'META800', 'ACET800I', 'FOSC12000I', 'GENTFORT', 'FLUP25', 'PROC25R', 'DIVA125', 'DRON10', 'THEO80LQ', 'VINC2I', 'TACR1', 'TRIAM.025/45O', 'ACET25T', 'CHOL750', 'ESTR0.025T', 'NORMOCARB', 'FELB400', 'SALM65H', 'INDO50R', 'BERA8I', 'RENV800', 'NCEFXIV', 'ARIP2', 'TROP1ES', 'ALTE50I', 'LEVE500', 'NS10I', 'TETR250', 'KCL10PM', 'BORT3.5SC', 'HPGB5PB', 'GEMF600', 'METO10', 'OXA2I', 'ETOM2I', 'ANUSHC', 'METRODESN', 'METR500PM', 'ARSE10I', 'COLEPWD', 'PAREIND2', 'BUP01ED', 'RETR200I', 'CORTIND', 'ENOX60I', 'DEXAM20I', 'CARV3125', 'MORP15I', 'NEOSYRSL', 'BETAVAL0.1/15C', 'DOLA12.5I', 'LIDX15C', 'NORE800SYR', 'BACTDS', 'TACR0.5SM', 'DABI75', 'BARI2/450L', 'STAN2', 'LEUC100I', 'OCTR100I', 'TEMO20', 'ISRA25', 'MYCO150', 'DIGO25', 'NICA40PM', 'NEOFOL', 'PROP80', 'ETRA100', 'THOR25I', 'MEPE100AMP', 'ASTR5I', 'GAMML30I', 'DOMEPKT', 'CISA10I', 'PAPA30I', 'DICL10', 'CHPR100', 'NEOSPIR', 'NEVI200', 'CAS30L', 'DESM0.1', 'KETA10050', 'ESMO100I', 'ARTTS', 'BETAQL', 'BUME25I', 'PHYT1L', 'NEOFLUC10L', 'METR500I', 'METR75G', 'PROM25I', 'PILO4ES', 'BAG', 'MORPCONC', 'DEX50SY', 'ZONI10L', 'SODI3N', 'DICL500', 'AMP', 'PROT250I', 'CELE100', 'DAU50I', 'RALT400', 'CLAR500SYR', 'HEP10I', 'DESI25', 'CVVH3556I', 'CEFI100L', 'FLUC250', 'LEVO112', 'MTX1000I', 'TOLT1', 'CASP70DESEN', 'ACYC800', 'NCEFZIV', 'IMIP10', 'MAFE8.5C', 'MPREDSYR', 'VANC1I', 'PEMO375', 'CLOB5', 'OLAN10I', 'WARF25', 'ENOX120I', 'CARB200XR', 'MYC30O', 'NMORP', 'RIBAV6I', 'ALTE2IV', 'TETA250I', 'NS100G', 'PROG8G', 'MPRED80I', 'LAMO100BN', 'KAOP180L', 'PB15', 'DIAZE5I', 'PHEN5I', 'FERRG325', 'GLUC1I', 'OLSA250', 'LOXA5', 'FURO20', 'CLOR375', 'CIPR200I', 'HYDR2.5ES', 'KAYEX50R', 'POLYEO', 'DERMGEL', 'MEGE400L', 'HPGB5I', 'canc70i', 'HYDR25O', 'KETA100/50NS', 'FURO20I', 'BIAX250', 'DEGA120I', 'PEGI180I', 'IRON100I', 'NSSYR10', 'METF750XR', 'TACR0.5', 'ACET650R', 'VITD1000', 'CARB100T', 'NEOLEUCL', 'METH10I', 'FOLI5I', 'ARIP25HT', 'GLYC1R', 'KASE40I', 'EMTR10L', 'MEDR10', 'DOXA5I', 'MAXIEO', 'METH1I', 'NEOS0.5I', 'LEVE1000L', 'LIDO2000I', 'BUSP10', 'F2B1ED', 'CLIN300L', 'MEGE20', 'CHO1I', 'HYD25PCA', 'BUSP5', 'DURA', 'ROSU5', 'CHOL500', 'QUIN300', 'CYAN1I', 'VANC1F', 'FENT100T', 'CISP50I', 'METH36', 'HALO05', 'VANC250PR', 'THIX2', 'LITH300C', 'CYAN100', 'PRAS10', 'BETAVAL.1/45O', 'TOPI200', 'SINE25250', 'PHENY100L', 'SW3000I', 'CARB100CR', 'FLEET', 'ACET120R', 'DILT60', 'FLUO20L', 'PALINEO', 'DDAV4I', 'AMPOBORAL', 'CLAD100I', 'FAMO20SM', 'CACL1000I', 'NACL23.4I', 'ASA81', 'ALPR5', 'DEFI80I', 'PILO1O', 'CELE200', 'GLYCO4I', 'SNLR260C', 'FLUC50', 'ACYC400', 'EPO3I', 'NORE8/250', 'CARV625', 'NEOHYDP', 'TORS20', 'DEXA100I', 'LEVO5ES', 'CYCL10', 'NEOCHLO', 'MPRED16', 'EFAV600', 'CAGLMEQ', 'ALLO20S', 'MPRED2', 'TOBRFES', 'WARF5', 'TRMP100', 'LID2PREMIX', 'PENT10I', 'RAMI2.5', 'CAPT25', 'DILT25I', 'LANT500', 'BACTDES6', 'MELO7.5', 'NFAT5', 'IBUT1I', 'MORP2-8', 'NDEPHBAG100', 'VANC.0125L', 'GEOD20', 'MCT4L', 'IMAT400L', 'CLOP300', 'PRED1ES', 'ERYT2S', 'IRON25I', 'NELF250', 'APRE80', 'BENA5', 'VIAL300', 'OXAC150', 'TERC.4C', 'TAXO80I', 'ZITHR250', 'MIRT30', 'ONDAN4I', 'MICO2C', 'CALCITVITD', 'DEFI80IND', 'DESI10', 'DROT20I', 'SILD50', 'PYRI50', 'EPOP1.5I', 'ROPI25', 'BACICAP', 'AMP500I', 'VERA120SR', 'GAMM10I', 'PROZ20', 'TERB5', 'SARG500I', 'SINM25100', 'FINA1L', 'SCOP15P', 'PANCEC', 'RIBA1I', 'VAL250L', 'ZENA200I', 'HYDZ10', 'MITO5I', 'AMPIDESEN', 'TRIA0.1O', 'COZAA50', 'LIDPF1I', 'VZIG625I', 'CLIN9000I', 'HALD5I', 'METE2I', 'NATH25I', 'NEOLORAZL', 'AZTR1000ID', 'PHENY100', 'DEXR500I', 'CIPR500', 'RIFA550', 'FLUT220H', 'ESTR625', 'NEFA150', 'FAMO20I', 'ENO3I', 'METD5', 'DONE5', 'NEOPENG2I', 'DEXA75', 'GEOD80', 'BETA10', 'CEFA1000I', 'ENOX100I', 'THRO10T', 'MYCO1000L', 'DEXA2', 'INDO25L', 'HEPA10SYR', 'HYDRSYR', 'ENOX80I', 'ZILE600', 'NEOSPIR4I', 'MORP50PCA', 'VIAL', 'SELE40I', 'MAGN50I', 'RASB1.5I', 'CLOPIND', 'AMOX875L', 'NEVI50L', 'CLAR250L', 'GLYO65OT', 'OLAN0.5L', 'HYDR1O', 'LEVO200I', 'PIRO20', 'LEUP22.5I', 'IMIG400I', 'METO100', 'PACL300NO', 'METO10L', 'DUPLEX50', 'DASA100', 'D5W', 'HUM100I', 'H1B1ED', 'FLUP5L', 'EPO40I', 'EPIPEN0.3I', 'MEPE100I', 'SUNI25', 'NS250G', 'ZIDO100', 'LEUP45I', 'VERA120', 'AZTR2FB', 'PALINEO50I', 'METD10', 'ASPI81INDP', 'NTG100I', 'KAYE15P', 'ADAL90', 'KPHO45I', 'NEOFER', 'DOXE10', 'FENT400L', 'ROSU10', 'CLON1I', 'PRISMACA5000', 'PROM25R', 'TOBR1200I', 'PARO25CR', 'TOCA5O', 'HYDRO4I', 'BISA10R', 'HC100R', 'ADAL60', 'CLOFPLACIND', 'PSND', 'ETHA25', 'CHLOR5', 'RITU500DESEN', 'THIO25', 'PRED25', 'MYCO750L', 'ALBU4R', 'OCTR30I', 'PHYT10IVPB', 'ZILE600CR', 'PALI3', 'CYTO2500I', 'FLUC400L', 'DICL25', 'BUPR3I', 'MYCO250', 'DANT20I', 'ACGEL80I', 'SEND5SYR', 'VANC.00125L', 'QUIN5', 'DIAZ2', 'SYR60ML', 'RIS1L', 'TRIF1', 'VANC5I', 'NICA20', 'SPIR100', 'BEND100I', 'TRIA0.1/80O', 'LIDOVISC', 'ALEM10I', 'ASCO500I', 'NADO40', 'BUME.5I', 'AUGM100L', 'ERYT500I', 'FURO100I', 'PRAM0.5', 'DTAP', 'PENI1F', 'ATR1EO', 'BUPR100', 'FENO145', 'MORP2-5', 'COLL30O', 'ILOP2', 'CHPR250', 'LEUP3.75I', 'LAMO100XR', 'VENL375', 'INTR20/500I', 'PHEN100IVPB', 'CEFX1F', 'PHEN100I', 'VANC500PR', 'METO5I', 'CETU100I', 'MTX50I', 'ADDE5XR', 'ALPR25', 'IN73HI', 'ETHA400', 'GLYCPED12', 'NEUT1I', 'VANC250L', 'CEPH50L', 'PORF75I', 'ACETSP', 'ADEN6I', 'METYR250', 'TACR1SM', 'LISI20', 'FENO48', 'MYCO250L', 'DIVA1750L', 'VENF25', 'ASPARSYR', 'PANC16', 'MINO10', 'BAG3000mL', 'RIZA10', 'BORT3.5I', 'MEDR150I', 'COLC0.3HT', 'DILT90SR', 'FAMOT20I', 'CALCG2/100NS', 'KETAIND', 'TACR1BN', 'IMIP50', 'BUTO1N', 'PHYS2I', 'FOSP100I', 'DROP5I', 'SULF10EO', 'PROP1PB', 'LABE100', 'FENT500I', '1/4NS100', 'DISO150CR', 'SYRINGE5', 'CLOT10', 'CYAN50', 'GLUT5', 'OXYC5', 'FENO160', 'BECA15G', 'AMPR150', 'CEFTR1000I', 'AZA100I', 'VERA240SR', 'QUIN324', 'GLUC2.5XL', 'NTG10P', 'CALC25', 'AMIN100', 'TOBR12I', 'GAST30L', 'NEOFAT', 'NITR30L', 'GABA400', 'ATRO1ES', 'CREON12', 'NEOR100IND', 'PENI250', 'PROP150', 'MPREDDESEN', 'THEO300', 'MORP10R', 'DARU300', 'RITA5', 'HUMA75/25I', 'CHLO20I', 'TIAZ360', 'MICO3SUPP', 'ADVA100/50H', 'GENT120FD', 'MORPIM', 'SUCC20I', 'TORS5', 'CEPACOL1', 'SULF200', 'ZONI25', 'DIVA250ER', 'DORZ2S', 'MORP4-8', 'GLIM4', 'OXYC40', 'CALC500', 'CROM4O', 'DRAM50', 'INTE9I', 'NACL234', 'LISI10', 'CEFO1F', 'HURRSPRAY', 'ISOS10', 'METH40IVPB', 'AMPI250', 'SODI65S', 'NIAC100', 'ALPR1', 'ONDA40I', 'SUMA25', 'EPO2I', 'APAP500', 'CEFA500I', 'CILL50', 'GABA300', 'LIOT50', 'CHLOR250', 'PENT250', 'MGBASE500I', 'ARGA250I', 'AUGM0.125L', 'LACR7O', 'ATGAM', 'FLEETEN', 'SIRO5L', 'CIDO75I', 'DOXA8T', 'SW10I', 'CYCL0', 'PYRI100', 'DURA100I', 'PRED1', 'THEO200', 'LAMI150', 'CARB300ER', 'DAKINQS', 'SODCIT$/500I', 'BACT10I', 'LACO100L', 'MBLHCV1IND', 'CLOP0.5L', 'VORI50', 'SERT25', 'RISP025', 'NICO4L', 'OXCA600', 'TRET40L', 'WARF10', 'CYTO5T', 'TRIVIT', 'CHTH25', 'FLUM500I', 'RISP50I', 'OXAC1F', 'ACET.125T', 'PENV500', 'SPIR25', 'EPO3INESRD', 'CARB200ER', 'SARG250I', 'SIMU20I', 'IMDU60', 'QUIN200', 'ACAM333', 'NACL3IRR', 'METH10L', 'RANI150', 'OMEP20S', 'MINX25', 'CLOP75', 'OXYCO5L', 'MESADESEN2', 'METHCONC', 'VALP250L', 'CSA25', 'BENA20', 'PHEN15', 'FOSI20', 'METH10', 'HYDR3R', 'KETR30I', 'METH1000I', 'TIMO5ES', 'NEOPHEN', 'NFAT4', 'NICO21P', 'NABC50S', 'CLIN600PM', 'HALD100I', 'SALS500T', 'PERP8L', 'CVVH', 'BUPI0.1EPD', 'HEPVAC', 'GENT100PM', 'NACIND', 'METAMOR', 'SSD400C', 'DASA70', 'AZAC100I', 'GLUC5XL', 'LCARN330', 'PREV60TP', 'PROC50', 'CLOT130C', 'DONE10', 'MVITH', 'PROP60', 'MICA100I', 'CAFFCIT60I', 'VINC2SYR', 'BETH25', 'CARM100I', 'CHOL4G', 'PERP2', 'ERGO50', 'BEVA400I', 'RIMA100', 'CLON5', 'D10W250', 'ALBU3H', 'NITRO1O', 'DOXE2I', 'CIME300', 'PANT40I', 'MPRED4', 'DOBU250I', 'GEM1I', 'LOMO', 'PYRI100I', 'CLOB05C', 'DISU250', 'CEPH250S', 'ERLO25', 'MESADESEN6']

In [6]:
len(theirs), len(mine)

(2841, 2841)

In [9]:
def compare(aa, bb):
    a = sorted(aa)
    b = sorted(bb)
    if (len(a) != len(b)):
        print("not the same length", len(a), len(b))
    l = min(len(a), len(b))
    for i in range(l):
        if (a[i] != b[i]):
            print("first different element:", i, a[i], b[i])
            return
    print("good")

In [10]:
compare(theirs, mine)

good
